# Clinical Synthetic Data Generation Framework

This notebook explores the performance of the following Synthetic Table Generation Methods

- **CTGAN** (Conditional Tabular GAN)
- **CTAB-GAN** (Conditional Tabular GAN with advanced preprocessing)
- **CTAB-GAN+** (Enhanced version with WGAN-GP losses, general transforms, and improved stability)
- **GANerAid** (Custom implementation)
- **CopulaGAN** (Copula-based GAN)
- **TVAE** (Variational Autoencoder)

- Section 1 prepares the environment and sources setup.py.
- Section 2 reads in the dataset and produces an initial suite of EDA. 
  - 2.1.1 - user needs to adapt for incoming dataset
  - 2.1.2 - user should review to ensure target colummns and categorical columns are properly identified
  - 2.1.3 - user should confirm that data has loaded properly
  - 2.1.4 - if your data has missing values, MICE is employed; user should review
  - CHUNK_2_1_4_C: This code samples 500 rows to be used in rest of notebook for purposes of testing; comment out this code for production.
  - 2.1.5 - Exploratory data analysis
- Section 3 demonstrates the performance of each metholodogy with some default hyperparameters. 
   - 3.1.1-3.1.6 Subsections for each method
   - 3.2 batch processing of figures/tables from section 3 
- Section 4 runs hyperparameter optimization. 
   - 4.1.1-4.1.6 Subsections for each method
   - 4.2 batch processing of figures/tables from section 4 describing the optimization process 
- Section 5 re-runs each model with their respective best hyperparameters. 
   - 5.1.1-5.1.6 re-run each model using the best hyperparameters identified in Section 4.1.1-4.1.6, resp.
   - 5.2 batch processing of figures/tables from Section 5.


Refer to readme.md, doc\Model-descriptions.md, doc\Objective-function.md.

## 1 Setup and Configuration

In [1]:
# Code Chunk ID: CHUNK_1_0_0_A - Import Setup Module
# Import all functionality from setup.py
from setup import *

print("🎯 SETUP MODULE IMPORTED SUCCESSFULLY!")
print("="*60)

Session timestamp captured: 2025-09-23
[OK] Essential data science libraries imported successfully!
Detected sklearn 1.7.1 - applying compatibility patch...
INFO: Applying sklearn compatibility patches for version 1.7.1
Global sklearn compatibility patch applied successfully
CTAB-GAN imported successfully from ./CTAB-GAN
[OK] CTAB-GAN+ detected and available
Session timestamp captured: 2025-09-23
[OK] Essential data science libraries imported successfully!
Detected sklearn 1.7.1 - applying compatibility patch...
INFO: Applying sklearn compatibility patches for version 1.7.1
Global sklearn compatibility patch applied successfully
CTAB-GAN imported successfully
[OK] CTAB-GAN+ detected and available
[OK] GANerAidModel imported successfully from src.models.implementations
[OK] All required libraries imported successfully
[OK] Comprehensive data quality evaluation function loaded!
[OK] Batch evaluation system loaded!
[OK] Enhanced objective function v2 with DYNAMIC TARGET COLUMN support def

## 2 Data Loading and Pre-processing

#### 2.1.1 USER ATTENTION NEEDED

Adapt this for your incoming dataset.

In [2]:
# Code Chunk ID: CHUNK_2_1_1_A
# =================== USER CONFIGURATION ===================
# 📝 CONFIGURE YOUR DATASET: Update these settings for your data
DATA_FILE = 'data/Pakistani_Diabetes_Dataset.csv'  # Path to your CSV file
TARGET_COLUMN = 'Outcome'                          # Name of your target/outcome column

# 🔧 DATASET IDENTIFIER (for results folder naming)
# Option 1: Manual override (recommended for consistent naming)
DATASET_IDENTIFIER_OVERRIDE = 'pakistani-diabetes-dataset'  # Set to None for auto-detection

# 🔧 OPTIONAL ADVANCED SETTINGS (Auto-detected if left empty)
CATEGORICAL_COLUMNS = ['Gender', 'Rgn']            # List categorical columns or leave empty for auto-detection
MISSING_STRATEGY = 'median'                        # Options: 'mice', 'drop', 'median', 'mode'
DATASET_NAME = 'Pakistani Diabetes Dataset'       # Descriptive name for your dataset

# 🚨 IMPORTANT: Verify these settings match your dataset before running!
print(f"📊 Configuration Summary:")
print(f"   Dataset: {DATASET_NAME}")
print(f"   File: {DATA_FILE}")
print(f"   Target: {TARGET_COLUMN}")
print(f"   Manual ID Override: {DATASET_IDENTIFIER_OVERRIDE}")
print(f"   Categorical: {CATEGORICAL_COLUMNS}")
print(f"   Missing Data Strategy: {MISSING_STRATEGY}")

# Load and prepare the dataset
data_file = DATA_FILE
target_column = TARGET_COLUMN

print(f"\n🔍 Loading dataset from: {data_file}")

try:
    data = pd.read_csv(data_file)
    print(f"✅ Dataset loaded successfully!")
    print(f"📊 Original shape: {data.shape}")
    
    # Set up dataset identifier with manual override option
    import setup
    
    # Use manual override if provided, otherwise auto-extract from filename
    if DATASET_IDENTIFIER_OVERRIDE:
        dataset_identifier = DATASET_IDENTIFIER_OVERRIDE
        print(f"📁 Using manual dataset identifier: {dataset_identifier}")
    else:
        dataset_identifier = setup.extract_dataset_identifier(data_file)
        print(f"📁 Auto-extracted dataset identifier: {dataset_identifier}")
    
    # Set global variables for setup module
    setup.DATASET_IDENTIFIER = dataset_identifier
    setup.CURRENT_DATA_FILE = data_file
    
    # Also set notebook global for consistency
    DATASET_IDENTIFIER = dataset_identifier
    
    print(f"✅ Dataset identifier set: {DATASET_IDENTIFIER}")
    print(f"📅 Session timestamp: {setup.SESSION_TIMESTAMP}")
    print(f"🗂️  Results will be saved to: results/{DATASET_IDENTIFIER}/")
    
except FileNotFoundError:
    print(f"❌ Error: File not found at {data_file}")
    print("   Please check the DATA_FILE path in your configuration above")
    print("   Current working directory:", os.getcwd())
    data = None

except Exception as e:
    print(f"❌ Error loading dataset: {e}")
    data = None

if data is not None:
    print(f"\n📋 Dataset Info:")
    print(f"   • Shape: {data.shape}")
    print(f"   • Columns: {list(data.columns)}")
    
    # Validate target column exists
    if target_column not in data.columns:
        print(f"\n❌ WARNING: Target column '{target_column}' not found!")
        print(f"   Available columns: {list(data.columns)}")
        print("   Please update TARGET_COLUMN in the configuration above")
    else:
        print(f"   • Target column '{target_column}' found ✅")
        print(f"   • Target distribution: {data[target_column].value_counts().to_dict()}")
    
    # Check for missing values
    missing_counts = data.isnull().sum()
    if missing_counts.sum() > 0:
        print(f"\n⚠️  Missing values detected:")
        for col, count in missing_counts[missing_counts > 0].items():
            print(f"   • {col}: {count} missing ({count/len(data)*100:.1f}%)")
    else:
        print(f"\n✅ No missing values detected")
else:
    print("\n❌ Dataset loading failed - please fix the configuration and try again")

📊 Configuration Summary:
   Dataset: Pakistani Diabetes Dataset
   File: data/Pakistani_Diabetes_Dataset.csv
   Target: Outcome
   Manual ID Override: pakistani-diabetes-dataset
   Categorical: ['Gender', 'Rgn']
   Missing Data Strategy: median

🔍 Loading dataset from: data/Pakistani_Diabetes_Dataset.csv
✅ Dataset loaded successfully!
📊 Original shape: (912, 19)
📁 Using manual dataset identifier: pakistani-diabetes-dataset
✅ Dataset identifier set: pakistani-diabetes-dataset
📅 Session timestamp: 2025-09-23
🗂️  Results will be saved to: results/pakistani-diabetes-dataset/

📋 Dataset Info:
   • Shape: (912, 19)
   • Columns: ['Age', 'Gender', 'Rgn ', 'wt', 'BMI', 'wst', 'sys', 'dia', 'his', 'A1c', 'B.S.R', 'vision', 'Exr', 'dipsia', 'uria', 'Dur', 'neph', 'HDL', 'Outcome']
   • Target column 'Outcome' found ✅
   • Target distribution: {1: 486, 0: 426}

✅ No missing values detected


The code defines utilities for column name standardization and dataset analysis using the pandas library in Python. It includes functions to clean and normalize column names, identify the target variable, categorize column types, and validate dataset configurations. These functions enhance data preprocessing by ensuring consistency and integrity, making it easier to manage various data types and handle potential issues like missing values. Overall, they provide a structured approach for effective dataset analysis.

#### 2.1.2 Column Name Standardization and Dataset Analysis Utilities

In [3]:
# Code Chunk ID: CHUNK_2_1_2_A
# Column Name Standardization and Dataset Analysis Utilities
import re
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple, Any

def standardize_column_names(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    # Create mapping of old to new column names
    name_mapping = {}
    
    for col in df.columns:
        # Remove special characters and normalize
        new_name = re.sub(r'[^\w\s]', '', str(col))  # Remove special chars
        new_name = re.sub(r'\s+', '_', new_name.strip())  # Replace spaces with underscores
        new_name = new_name.lower()  # Convert to lowercase
        
        # Handle duplicate names
        if new_name in name_mapping.values():
            counter = 1
            while f"{new_name}_{counter}" in name_mapping.values():
                counter += 1
            new_name = f"{new_name}_{counter}"
            
        name_mapping[col] = new_name
    
    # Rename columns
    df = df.rename(columns=name_mapping)
    
    print(f"🔄 Column Name Standardization:")
    for old, new in name_mapping.items():
        if old != new:
            print(f"   '{old}' → '{new}'")
    
    return df, name_mapping

def detect_target_column(df: pd.DataFrame, target_hint: str = None) -> str:
    """
    Detect the target column in the dataset.
    
    Args:
        df: Input dataframe
        target_hint: User-provided hint for target column name
        
    Returns:
        Name of the detected target column
    """
    # Common target column patterns
    target_patterns = [
        'target', 'label', 'class', 'outcome', 'result', 'diagnosis', 
        'response', 'y', 'dependent', 'output', 'prediction'
    ]
    
    # If user provided hint, try to find it first
    if target_hint:
        # Try exact match (case insensitive)
        for col in df.columns:
            if col.lower() == target_hint.lower():
                print(f"✅ Target column found: '{col}' (user specified)")
                return col
        
        # Try partial match
        for col in df.columns:
            if target_hint.lower() in col.lower():
                print(f"✅ Target column found: '{col}' (partial match to '{target_hint}')")
                return col
    
    # Auto-detect based on patterns
    for pattern in target_patterns:
        for col in df.columns:
            if pattern in col.lower():
                print(f"✅ Target column auto-detected: '{col}' (pattern: '{pattern}')")
                return col
    
    # If no pattern match, check for binary columns (likely targets)
    binary_cols = []
    for col in df.columns:
        unique_vals = df[col].dropna().nunique()
        if unique_vals == 2:
            binary_cols.append(col)
    
    if binary_cols:
        target_col = binary_cols[0]  # Take first binary column
        print(f"✅ Target column inferred: '{target_col}' (binary column)")
        return target_col
    
    # Last resort: use last column
    target_col = df.columns[-1]
    print(f"⚠️ Target column defaulted to: '{target_col}' (last column)")
    return target_col

def analyze_column_types(df: pd.DataFrame, categorical_hint: List[str] = None) -> Dict[str, str]:
    """
    Analyze and categorize column types.
    
    Args:
        df: Input dataframe
        categorical_hint: User-provided list of categorical columns
        
    Returns:
        Dictionary mapping column names to types ('categorical', 'continuous', 'binary')
    """
    column_types = {}
    
    for col in df.columns:
        # Skip if user explicitly specified as categorical
        if categorical_hint and col in categorical_hint:
            column_types[col] = 'categorical'
            continue
            
        # Analyze column characteristics
        non_null_data = df[col].dropna()
        unique_count = non_null_data.nunique()
        total_count = len(non_null_data)
        
        # Determine type based on data characteristics
        if unique_count == 2:
            column_types[col] = 'binary'
        elif df[col].dtype == 'object' or unique_count < 10:
            column_types[col] = 'categorical'
        elif df[col].dtype in ['int64', 'float64'] and unique_count > 10:
            column_types[col] = 'continuous'
        else:
            # Default based on uniqueness ratio
            uniqueness_ratio = unique_count / total_count
            if uniqueness_ratio < 0.1:
                column_types[col] = 'categorical'
            else:
                column_types[col] = 'continuous'
    
    return column_types

def validate_dataset_config(df: pd.DataFrame, target_col: str, config: Dict[str, Any]) -> bool:
    """
    Validate dataset configuration and provide warnings.
    
    Args:
        df: Input dataframe
        target_col: Target column name
        config: Configuration dictionary
        
    Returns:
        True if validation passes, False otherwise
    """
    print(f"\n🔍 Dataset Validation:")
    
    valid = True
    
    # Check if target column exists
    if target_col not in df.columns:
        print(f"❌ Target column '{target_col}' not found in dataset!")
        print(f"   Available columns: {list(df.columns)}")
        valid = False
    else:
        print(f"✅ Target column '{target_col}' found")
    
    # Check dataset size
    if len(df) < 100:
        print(f"⚠️ Small dataset: {len(df)} rows (recommend >1000 for synthetic data)")
    else:
        print(f"✅ Dataset size: {len(df)} rows")
    
    # Check for missing data
    missing_pct = (df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100
    if missing_pct > 20:
        print(f"⚠️ High missing data: {missing_pct:.1f}% (recommend MICE imputation)")
    elif missing_pct > 0:
        print(f"🔍 Missing data: {missing_pct:.1f}% (manageable)")
    else:
        print(f"✅ No missing data")
    
    return valid

print("✅ Dataset analysis utilities loaded successfully!")

✅ Dataset analysis utilities loaded successfully!


#### 2.1.3 Load and Analyze Dataset with Generalized Configuration

This code loads and analyzes a dataset using a specified configuration. It imports necessary libraries, attempts to read a CSV file, and standardizes the column names while allowing for potential updates to the target column. The script detects the target column, analyzes data types, and validates the dataset configuration, providing a summary of the dataset’s shape and missing values. Finally, it stores metadata about the dataset for future reference.

In [4]:
# Code Chunk ID: CHUNK_2_1_3_A
# =================== USER CONFIGURATION ===================
# 📝 CONFIGURE YOUR DATASET: Update these settings for your data
DATA_FILE = 'data/Pakistani_Diabetes_Dataset.csv'  # Path to your CSV file
TARGET_COLUMN = 'Outcome'                          # Name of your target/outcome column

# 🔧 DATASET IDENTIFIER (for results folder naming)
# Option 1: Manual override (recommended for consistent naming)
DATASET_IDENTIFIER_OVERRIDE = 'pakistani-diabetes-dataset'  # Set to None for auto-detection

# 🔧 OPTIONAL ADVANCED SETTINGS (Auto-detected if left empty)
CATEGORICAL_COLUMNS = ['Gender', 'Rgn']            # List categorical columns or leave empty for auto-detection
MISSING_STRATEGY = 'median'                        # Options: 'mice', 'drop', 'median', 'mode'
DATASET_NAME = 'Pakistani Diabetes Dataset'       # Descriptive name for your dataset

# 🚨 IMPORTANT: Verify these settings match your dataset before running!
print(f"📊 Configuration Summary:")
print(f"   Dataset: {DATASET_NAME}")
print(f"   File: {DATA_FILE}")
print(f"   Target: {TARGET_COLUMN}")
print(f"   Manual ID Override: {DATASET_IDENTIFIER_OVERRIDE}")
print(f"   Categorical: {CATEGORICAL_COLUMNS}")
print(f"   Missing Data Strategy: {MISSING_STRATEGY}")

# Load and prepare the dataset
data_file = DATA_FILE
target_column = TARGET_COLUMN

print(f"\n🔍 Loading dataset from: {data_file}")

try:
    data = pd.read_csv(data_file)
    print(f"✅ Dataset loaded successfully!")
    print(f"📊 Original shape: {data.shape}")
    
    # Set up dataset identifier with manual override option
    import setup
    
    # Use manual override if provided, otherwise auto-extract from filename
    if DATASET_IDENTIFIER_OVERRIDE:
        dataset_identifier = DATASET_IDENTIFIER_OVERRIDE
        setup.DATASET_IDENTIFIER = DATASET_IDENTIFIER_OVERRIDE
        setup.CURRENT_DATA_FILE = data_file
        print(f"📁 Using manual dataset identifier: {dataset_identifier}")
    else:
        dataset_identifier = setup.extract_dataset_identifier(data_file)
        setup.DATASET_IDENTIFIER = dataset_identifier
        setup.CURRENT_DATA_FILE = data_file
        print(f"📁 Auto-extracted dataset identifier: {dataset_identifier}")
    
    # 🔧 CRITICAL FIX: Set global DATASET_IDENTIFIER for use in other chunks
    DATASET_IDENTIFIER = dataset_identifier  # This ensures other chunks can access it
    
    # 📁 NEW: Update RESULTS_DIR for organized file outputs using proper structure
    # Don't set a specific RESULTS_DIR here - let each section use get_results_path()
    # This ensures proper date/section structure like: results/dataset/2025-09-12/Section-2/
    RESULTS_DIR = f"results/{dataset_identifier}/"  # Base directory only
    
    print(f"✅ Dataset identifier set: {DATASET_IDENTIFIER}")
    print(f"📅 Session timestamp: {setup.SESSION_TIMESTAMP}")
    print(f"🗂️  Results will be saved to: results/{dataset_identifier}/")
    
except FileNotFoundError:
    print(f"❌ Error: File not found at {data_file}")
    print("   Please check the DATA_FILE path in your configuration above")
    print("   Current working directory:", os.getcwd())
    data = None

except Exception as e:
    print(f"❌ Error loading dataset: {e}")
    data = None

if data is not None:
    print(f"\n📋 Dataset Info:")
    print(f"   • Shape: {data.shape}")
    print(f"   • Columns: {list(data.columns)}")
    
    # Check if target column exists
    if target_column not in data.columns:
        print(f"\n❌ WARNING: Target column '{target_column}' not found!")
        print(f"   Available columns: {list(data.columns)}")
        print("   Please update TARGET_COLUMN in the configuration above")
    else:
        print(f"   • Target column '{target_column}' found ✅")
        print(f"   • Target distribution: {data[target_column].value_counts().to_dict()}")
    
    # Check for missing values
    missing_values = data.isnull().sum()
    if missing_values.sum() > 0:
        print(f"\n⚠️  Missing values detected:")
        for col, count in missing_values[missing_values > 0].items():
            print(f"   • {col}: {count} missing ({count/len(data)*100:.1f}%)")
    else:
        print(f"\n✅ No missing values detected")
else:
    print("\n❌ Dataset loading failed - please fix the configuration and try again")

📊 Configuration Summary:
   Dataset: Pakistani Diabetes Dataset
   File: data/Pakistani_Diabetes_Dataset.csv
   Target: Outcome
   Manual ID Override: pakistani-diabetes-dataset
   Categorical: ['Gender', 'Rgn']
   Missing Data Strategy: median

🔍 Loading dataset from: data/Pakistani_Diabetes_Dataset.csv
✅ Dataset loaded successfully!
📊 Original shape: (912, 19)
📁 Using manual dataset identifier: pakistani-diabetes-dataset
✅ Dataset identifier set: pakistani-diabetes-dataset
📅 Session timestamp: 2025-09-23
🗂️  Results will be saved to: results/pakistani-diabetes-dataset/

📋 Dataset Info:
   • Shape: (912, 19)
   • Columns: ['Age', 'Gender', 'Rgn ', 'wt', 'BMI', 'wst', 'sys', 'dia', 'his', 'A1c', 'B.S.R', 'vision', 'Exr', 'dipsia', 'uria', 'Dur', 'neph', 'HDL', 'Outcome']
   • Target column 'Outcome' found ✅
   • Target distribution: {1: 486, 0: 426}

✅ No missing values detected


This code provides advanced utilities for handling missing data using various strategies in Python. It includes functions to assess missing data patterns, apply Multiple Imputation by Chained Equations (MICE), visualize missing patterns, and implement different strategies for managing missing values. The `assess_missing_patterns` function analyzes and summarizes missing data, while `apply_mice_imputation` leverages an iterative imputer for numeric columns. The `visualize_missing_patterns` function creates visual representations of missing data, and the `handle_missing_data_strategy` function executes the chosen strategy, offering options like MICE, dropping rows, or filling with median or mode values. Collectively, these utilities facilitate effective management of missing data to improve dataset quality.

#### 2.1.4 Advanced Missing Data Handling with MICE

This code provides a comprehensive toolkit for analyzing, visualizing, and handling missing data in a pandas DataFrame using advanced and flexible approaches. It includes functions to assess the extent and patterns of missingness, visualize those patterns, and apply various imputation techniques. The centerpiece is the ability to perform Multiple Imputation by Chained Equations (MICE) using scikit-learn’s IterativeImputer with a RandomForestRegressor (for numerical features), which statistically estimates and fills in missing values based on all other feature relationships. For categorical variables, the code defaults to simpler mode imputation. Users can also choose alternative strategies like dropping rows (drop), filling with column medians (median), or filling with the most frequent value (mode). The code features detailed feedback and visual support with heatmaps and bar plots to help the user understand and monitor missing data patterns throughout the handling process. Together, these utilities help users robustly prepare data for machine learning or statistical analysis, reducing bias and maximizing data retention and utility.

In [5]:
# Code Chunk ID: CHUNK_2_1_4_A
# Advanced Missing Data Handling with MICE
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def assess_missing_patterns(df: pd.DataFrame) -> dict:
    """
    Comprehensive assessment of missing data patterns.
    
    Args:
        df: Input dataframe
        
    Returns:
        Dictionary with missing data analysis
    """
    missing_analysis = {}
    
    # Basic missing statistics
    missing_counts = df.isnull().sum()
    missing_percentages = (missing_counts / len(df)) * 100
    
    missing_analysis['missing_counts'] = missing_counts[missing_counts > 0]
    missing_analysis['missing_percentages'] = missing_percentages[missing_percentages > 0]
    missing_analysis['total_missing_cells'] = df.isnull().sum().sum()
    missing_analysis['total_cells'] = df.size
    missing_analysis['overall_missing_rate'] = (missing_analysis['total_missing_cells'] / missing_analysis['total_cells']) * 100
    
    # Missing patterns
    missing_patterns = df.isnull().value_counts()
    missing_analysis['missing_patterns'] = missing_patterns
    
    return missing_analysis

def apply_mice_imputation(df: pd.DataFrame, target_col: str = None, max_iter: int = 10, random_state: int = 42) -> pd.DataFrame:
    """
    Apply Multiple Imputation by Chained Equations (MICE) to handle missing data.
    
    Args:
        df: Input dataframe with missing values
        target_col: Target column name (excluded from imputation predictors)
        max_iter: Maximum number of imputation iterations
        random_state: Random state for reproducibility
        
    Returns:
        DataFrame with imputed values
    """
    print(f"🔧 Applying MICE imputation...")
    
    # Separate features and target
    if target_col and target_col in df.columns:
        features = df.drop(columns=[target_col])
        target = df[target_col]
    else:
        features = df.copy()
        target = None
    
    # Identify numeric and categorical columns
    numeric_cols = features.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = features.select_dtypes(include=['object', 'category']).columns.tolist()
    
    df_imputed = features.copy()
    
    # Handle numeric columns with MICE
    if numeric_cols:
        print(f"   Imputing {len(numeric_cols)} numeric columns...")
        numeric_imputer = IterativeImputer(
            estimator=RandomForestRegressor(n_estimators=10, random_state=random_state),
            max_iter=max_iter,
            random_state=random_state
        )
        
        numeric_imputed = numeric_imputer.fit_transform(features[numeric_cols])
        df_imputed[numeric_cols] = numeric_imputed
    
    # Handle categorical columns with mode imputation (simpler approach)
    if categorical_cols:
        print(f"   Imputing {len(categorical_cols)} categorical columns with mode...")
        for col in categorical_cols:
            mode_value = features[col].mode()
            if len(mode_value) > 0:
                df_imputed[col] = features[col].fillna(mode_value[0])
            else:
                # If no mode, fill with 'Unknown'
                df_imputed[col] = features[col].fillna('Unknown')
    
    # Add target back if it exists
    if target is not None:
        df_imputed[target_col] = target
    
    print(f"✅ MICE imputation completed!")
    print(f"   Missing values before: {features.isnull().sum().sum()}")
    print(f"   Missing values after: {df_imputed.isnull().sum().sum()}")
    
    return df_imputed

def visualize_missing_patterns(df: pd.DataFrame, title: str = "Missing Data Patterns") -> None:
    """
    Create visualizations for missing data patterns.
    
    Args:
        df: Input dataframe
        title: Title for the plot
    """
    missing_data = df.isnull()
    
    if missing_data.sum().sum() == 0:
        print("✅ No missing data to visualize!")
        return
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Missing data heatmap
    sns.heatmap(missing_data, 
                yticklabels=False, 
                cbar=True, 
                cmap='viridis',
                ax=axes[0])
    axes[0].set_title('Missing Data Heatmap')
    axes[0].set_xlabel('Columns')
    
    # Missing data bar chart
    missing_counts = missing_data.sum()
    missing_counts = missing_counts[missing_counts > 0]
    
    if len(missing_counts) > 0:
        missing_counts.plot(kind='bar', ax=axes[1], color='coral')
        axes[1].set_title('Missing Values by Column')
        axes[1].set_ylabel('Count of Missing Values')
        axes[1].tick_params(axis='x', rotation=45)
    else:
        axes[1].text(0.5, 0.5, 'No Missing Data', 
                    horizontalalignment='center', 
                    verticalalignment='center',
                    transform=axes[1].transAxes,
                    fontsize=16)
        axes[1].set_title('Missing Values by Column')
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

def handle_missing_data_strategy(df: pd.DataFrame, strategy: str, target_col: str = None) -> pd.DataFrame:
    """
    Apply the specified missing data handling strategy.
    
    Args:
        df: Input dataframe
        strategy: Strategy to use ('mice', 'drop', 'median', 'mode')
        target_col: Target column name
        
    Returns:
        DataFrame with missing data handled
    """
    print(f"\n🔧 Applying missing data strategy: {strategy.upper()}")
    
    if df.isnull().sum().sum() == 0:
        print("✅ No missing data detected - no imputation needed")
        return df.copy()
    
    if strategy.lower() == 'mice':
        return apply_mice_imputation(df, target_col)
    
    elif strategy.lower() == 'drop':
        print(f"   Dropping rows with missing values...")
        df_clean = df.dropna()
        print(f"   Rows before: {len(df)}, Rows after: {len(df_clean)}")
        return df_clean
    
    elif strategy.lower() == 'median':
        print(f"   Filling missing values with median/mode...")
        df_filled = df.copy()
        
        # Numeric columns: fill with median
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            if df[col].isnull().sum() > 0:
                median_val = df[col].median()
                df_filled[col] = df[col].fillna(median_val)
                print(f"     {col}: filled {df[col].isnull().sum()} values with median {median_val:.2f}")
        
        # Categorical columns: fill with mode
        categorical_cols = df.select_dtypes(include=['object', 'category']).columns
        for col in categorical_cols:
            if df[col].isnull().sum() > 0:
                mode_val = df[col].mode()
                if len(mode_val) > 0:
                    df_filled[col] = df[col].fillna(mode_val[0])
                    print(f"     {col}: filled {df[col].isnull().sum()} values with mode '{mode_val[0]}'")
        
        return df_filled
    
    elif strategy.lower() == 'mode':
        print(f"   Filling missing values with mode...")
        df_filled = df.copy()
        
        for col in df.columns:
            if df[col].isnull().sum() > 0:
                mode_val = df[col].mode()
                if len(mode_val) > 0:
                    df_filled[col] = df[col].fillna(mode_val[0])
                    print(f"     {col}: filled {df[col].isnull().sum()} values with mode '{mode_val[0]}'")
        
        return df_filled
    
    else:
        print(f"⚠️ Unknown strategy '{strategy}'. Using 'median' as fallback.")
        return handle_missing_data_strategy(df, 'median', target_col)

print("✅ Missing data handling utilities loaded successfully!")

✅ Missing data handling utilities loaded successfully!


⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️

Note that next chunk samples 500 rows

⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️

In [ ]:
# CHUNK_2_1_4_C
# This chunk samples 500 rows for quicker testing - remove for full dataset
data=data.sample(n=500, random_state=42)
data.shape
data.head()

,Age,Gender,Rgn,wt,BMI,wst,sys,dia,his,A1c,B.S.R,vision,Exr,dipsia,uria,Dur,neph,HDL,Outcome
649,56.0,1,1,56.0,27.50,40.0,120,90,1,9.4,225,0,30,0,1,10.0,0,43,1
761,34.0,1,1,63.0,27.50,39.0,110,90,1,5.0,205,1,0,1,1,4.0,0,47,1
545,23.0,0,0,55.0,34.00,30.0,120,80,1,5.3,134,0,0,0,0,0.0,0,48,0
367,25.0,1,0,75.0,27.51,36.0,110,80,0,5.0,90,1,15,0,0,0.0,0,43,0
361,54.0,0,1,55.0,22.70,30.0,130,90,0,4.9,89,1,30,0,0,0.0,0,53,0


#### 2.1.5 EDA
This code snippet provides an enhanced overview and analysis of a dataset. It generates basic statistics, including the dataset name, shape, memory usage, total missing values, missing percentage, number of duplicate rows, and counts of numeric and categorical columns. The results are organized into a dictionary called `overview_stats`, which is then iterated over to print each statistic in a formatted manner. Additionally, it sets up for displaying a sample of the data afterward.

In [7]:
# Code Chunk ID: CHUNK_2_1_5_A
# Enhanced Dataset Overview and Analysis
print("📋 COMPREHENSIVE DATASET OVERVIEW")
print("=" * 60)

# Basic statistics
overview_stats = {
    'Dataset Name': 'Breast Cancer Wisconsin (Diagnostic)',
    'Shape': f"{data.shape[0]} rows × {data.shape[1]} columns",
    'Memory Usage': f"{data.memory_usage(deep=True).sum() / 1024**2:.2f} MB",
    'Total Missing Values': data.isnull().sum().sum(),
    'Missing Percentage': f"{(data.isnull().sum().sum() / data.size) * 100:.2f}%",
    'Duplicate Rows': data.duplicated().sum(),
    'Numeric Columns': len(data.select_dtypes(include=[np.number]).columns),
    'Categorical Columns': len(data.select_dtypes(include=['object']).columns)
}

for key, value in overview_stats.items():
    print(f"{key:.<25} {value}")

📋 COMPREHENSIVE DATASET OVERVIEW
Dataset Name............. Breast Cancer Wisconsin (Diagnostic)
Shape.................... 500 rows × 19 columns
Memory Usage............. 0.08 MB
Total Missing Values..... 0
Missing Percentage....... 0.00%
Duplicate Rows........... 1
Numeric Columns.......... 19
Categorical Columns...... 0


In [8]:
# Code Chunk ID: CHUNK_2_1_5_B
# Enhanced Column Analysis - OUTPUT TO FILE
print("📊 DETAILED COLUMN ANALYSIS (SAVING TO FILE)")
print("=" * 50)

column_analysis = pd.DataFrame({
    'Column': data.columns,
    'Data_Type': data.dtypes.astype(str),
    'Unique_Values': [data[col].nunique() for col in data.columns],
    'Missing_Count': [data[col].isnull().sum() for col in data.columns],
    'Missing_Percent': [f"{(data[col].isnull().sum()/len(data)*100):.2f}%" for col in data.columns],
    'Min_Value': [data[col].min() if data[col].dtype in ['int64', 'float64'] else 'N/A' for col in data.columns],
    'Max_Value': [data[col].max() if data[col].dtype in ['int64', 'float64'] else 'N/A' for col in data.columns]
})

# Use new folder structure: results/dataset_identifier/YYYY-MM-DD/Section-2
results_path = get_results_path(DATASET_IDENTIFIER, 2)
os.makedirs(results_path, exist_ok=True)
csv_file = f'{results_path}/column_analysis.csv'
column_analysis.to_csv(csv_file, index=False)

print(f"📊 Column analysis table saved to {csv_file}")
print(f"📊 Analysis completed for {len(data.columns)} features")

📊 DETAILED COLUMN ANALYSIS (SAVING TO FILE)
📊 Column analysis table saved to results/pakistani-diabetes-dataset/2025-09-23/Section-2/column_analysis.csv
📊 Analysis completed for 19 features


This code conducts an enhanced analysis of the target variable within a dataset. It computes the counts and percentages of target classes, organizing the results into a DataFrame called `target_summary`, which distinguishes between benign and malignant classes if applicable. The class balance is assessed by calculating a balance ratio, with outputs indicating whether the dataset is balanced, moderately imbalanced, or highly imbalanced. If the specified target column is not found, it displays a warning and lists available columns in the dataset.

In [9]:
# Code Chunk ID: CHUNK_2_1_5_C
# Enhanced Target Variable Analysis - OUTPUT TO FILE
print("🎯 TARGET VARIABLE ANALYSIS (SAVING TO FILE)")
print("=" * 40)

if target_column in data.columns:
    target_counts = data[target_column].value_counts().sort_index()
    target_props = data[target_column].value_counts(normalize=True).sort_index() * 100
    
    target_summary = pd.DataFrame({
        'Class': target_counts.index,
        'Count': target_counts.values,
        'Percentage': [f"{prop:.1f}%" for prop in target_props.values],
        'Description': ['Benign (Non-cancerous)', 'Malignant (Cancerous)'] if len(target_counts) == 2 else [f'Class {i}' for i in target_counts.index]
    })
    
    # Use new folder structure: results/dataset_identifier/YYYY-MM-DD/Section-2
    results_path = get_results_path(DATASET_IDENTIFIER, 2)
    os.makedirs(results_path, exist_ok=True)
    csv_file = f'{results_path}/target_analysis.csv'
    target_summary.to_csv(csv_file, index=False)
    
    # Calculate class balance metrics
    balance_ratio = target_counts.min() / target_counts.max()
    
    # Save balance metrics to separate file
    balance_metrics = pd.DataFrame({
        'Metric': ['Class_Balance_Ratio', 'Dataset_Balance_Category'],
        'Value': [f"{balance_ratio:.3f}", 
                 'Balanced' if balance_ratio > 0.8 else 'Moderately Imbalanced' if balance_ratio > 0.5 else 'Highly Imbalanced']
    })
    balance_file = f'{results_path}/target_balance_metrics.csv'
    balance_metrics.to_csv(balance_file, index=False)
    
    print(f"📊 Target variable analysis saved to {csv_file}")
    print(f"📊 Class balance metrics saved to {balance_file}")
    print(f"📊 Class Balance Ratio: {balance_ratio:.3f}")
    print(f"📊 Dataset Balance: {'Balanced' if balance_ratio > 0.8 else 'Moderately Imbalanced' if balance_ratio > 0.5 else 'Highly Imbalanced'}")
    
else:
    print(f"⚠️ Warning: Target column '{target_column}' not found!")
    print(f"Available columns: {list(data.columns)}")

🎯 TARGET VARIABLE ANALYSIS (SAVING TO FILE)
📊 Target variable analysis saved to results/pakistani-diabetes-dataset/2025-09-23/Section-2/target_analysis.csv
📊 Class balance metrics saved to results/pakistani-diabetes-dataset/2025-09-23/Section-2/target_balance_metrics.csv
📊 Class Balance Ratio: 0.887
📊 Dataset Balance: Balanced


This code provides enhanced visualizations of feature distributions in a dataset. It retrieves numeric columns, excluding the target variable, and generates histograms for each numeric feature, displaying them in a grid layout. The histograms are enhanced with options for density, color, and grid lines to improve readability. If no numeric features are found, a warning message is displayed; otherwise, the generated plots give insights into the distributions of the numeric features in the dataset.

In [10]:
# Code Chunk ID: CHUNK_2_1_5_D
# Enhanced Feature Distribution Visualizations - OUTPUT TO FILE
print("📊 FEATURE DISTRIBUTION ANALYSIS (SAVING TO FILE)")
print("=" * 40)

# Turn off interactive mode to prevent figures from displaying in notebook
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
plt.ioff()  # Turn off interactive mode

# Get numeric columns excluding target
numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
if target_column in numeric_cols:
    numeric_cols.remove(target_column)

if numeric_cols:
    n_cols = min(3, len(numeric_cols))
    n_rows = (len(numeric_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows))
    # Use dataset name fallback for title
    dataset_name = DATASET_IDENTIFIER.title() if DATASET_IDENTIFIER else "Dataset"
    fig.suptitle(f'{dataset_name} - Feature Distributions', fontsize=16, fontweight='bold')
    
    # Handle different subplot configurations
    if n_rows == 1 and n_cols == 1:
        axes = [axes]
    elif n_rows == 1:
        axes = axes
    else:
        axes = axes.flatten()
    
    for i, col in enumerate(numeric_cols):
        if i < len(axes):
            # Enhanced histogram
            axes[i].hist(data[col], bins=30, alpha=0.7, color='skyblue', 
                        edgecolor='black', density=True)
            
            axes[i].set_title(f'{col}', fontsize=12, fontweight='bold')
            axes[i].set_xlabel(col)
            axes[i].set_ylabel('Density')
            axes[i].grid(True, alpha=0.3)
    
    # Remove empty subplots
    for j in range(len(numeric_cols), len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    
    # Use new folder structure: results/dataset_identifier/YYYY-MM-DD/Section-2
    results_path = get_results_path(DATASET_IDENTIFIER, 2)
    os.makedirs(results_path, exist_ok=True)
    plot_file = f'{results_path}/feature_distributions.png'
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    plt.close()  # Close the figure to free memory
    
    print(f"📊 Feature distribution plots saved to {plot_file}")
    print(f"📊 Distribution analysis completed for {len(numeric_cols)} numeric features")
else:
    print("⚠️ No numeric features found for visualization")

📊 FEATURE DISTRIBUTION ANALYSIS (SAVING TO FILE)
📊 Feature distribution plots saved to results/pakistani-diabetes-dataset/2025-09-23/Section-2/feature_distributions.png
📊 Distribution analysis completed for 18 numeric features


This code conducts an enhanced correlation analysis of features within a dataset. It calculates the correlation matrix for numeric columns and includes the target variable if it is numeric, displaying the results in a heatmap for better visualization. The analysis identifies correlations with the target variable, categorizing each feature based on its correlation strength (strong, moderate, or weak) and presenting the findings in a DataFrame. If there are insufficient numeric features, a warning message is displayed, indicating that correlation analysis cannot be performed.

In [11]:
# Code Chunk ID: CHUNK_2_1_5_E
# Enhanced Correlation Analysis - OUTPUT TO FILE
print("🔍 CORRELATION ANALYSIS (SAVING TO FILE)")
print("=" * 30)

# Turn off interactive mode to prevent figures from displaying in notebook
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
plt.ioff()  # Turn off interactive mode

if len(numeric_cols) > 1:
    # Include target in correlation if numeric
    cols_for_corr = numeric_cols.copy()
    if data[target_column].dtype in ['int64', 'float64']:
        cols_for_corr.append(target_column)
    
    correlation_matrix = data[cols_for_corr].corr()
    
    # Enhanced correlation heatmap
    fig, ax = plt.subplots(figsize=(10, 8))
    
    sns.heatmap(correlation_matrix, 
                annot=True, 
                cmap='RdBu_r',
                center=0, 
                square=True, 
                linewidths=0.5,
                fmt='.3f',
                ax=ax)
    
    # Use dataset name fallback for title
    dataset_name = DATASET_IDENTIFIER.title() if DATASET_IDENTIFIER else "Dataset"
    ax.set_title(f'{dataset_name} - Feature Correlation Matrix', 
              fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    
    # Use new folder structure: results/dataset_identifier/YYYY-MM-DD/Section-2
    results_path = get_results_path(DATASET_IDENTIFIER, 2)
    os.makedirs(results_path, exist_ok=True)
    heatmap_file = f'{results_path}/correlation_heatmap.png'
    plt.savefig(heatmap_file, dpi=300, bbox_inches='tight')
    plt.close()  # Close the figure to free memory
    
    # Save correlation matrix to CSV
    corr_matrix_file = f'{results_path}/correlation_matrix.csv'
    correlation_matrix.to_csv(corr_matrix_file)
    
    print(f"🔍 Correlation heatmap saved to {heatmap_file}")
    print(f"🔍 Correlation matrix saved to {corr_matrix_file}")
    
    # Correlation with target analysis
    if target_column in correlation_matrix.columns:
        print("\n🔍 CORRELATIONS WITH TARGET VARIABLE (SAVING TO FILE)")
        print("=" * 45)
        
        target_corrs = correlation_matrix[target_column].abs().sort_values(ascending=False)
        target_corrs = target_corrs[target_corrs.index != target_column]
        
        corr_analysis = pd.DataFrame({
            'Feature': target_corrs.index,
            'Absolute_Correlation': target_corrs.values,
            'Raw_Correlation': [correlation_matrix.loc[feat, target_column] for feat in target_corrs.index],
            'Strength': ['Strong' if abs(corr) > 0.7 else 'Moderate' if abs(corr) > 0.3 else 'Weak' 
                        for corr in target_corrs.values]
        })
        
        # Save correlation analysis to CSV instead of displaying
        corr_analysis_file = f'{results_path}/target_correlations.csv'
        corr_analysis.to_csv(corr_analysis_file, index=False)
        
        print(f"🔍 Target correlation analysis saved to {corr_analysis_file}")
        print(f"📊 Correlation analysis completed for {len(target_corrs)} features")
    
else:
    print("⚠️ Insufficient numeric features for correlation analysis")

🔍 CORRELATION ANALYSIS (SAVING TO FILE)
🔍 Correlation heatmap saved to results/pakistani-diabetes-dataset/2025-09-23/Section-2/correlation_heatmap.png
🔍 Correlation matrix saved to results/pakistani-diabetes-dataset/2025-09-23/Section-2/correlation_matrix.csv

🔍 CORRELATIONS WITH TARGET VARIABLE (SAVING TO FILE)
🔍 Target correlation analysis saved to results/pakistani-diabetes-dataset/2025-09-23/Section-2/target_correlations.csv
📊 Correlation analysis completed for 18 features


This code sets up global configuration variables for consistent evaluation across model evaluations. It checks for the existence of required variables, such as `data` and `target_column`, and raises an error if they are not defined. The code establishes global constants for the target column, results directory, and a copy of the original data while defining categorical columns, excluding the target. It then creates the results directory if it does not already exist and verifies that all necessary global variables are present, providing feedback on the setup's success.

In [12]:
# Code Chunk ID: CHUNK_2_1_5_F
# ============================================================================
# GLOBAL CONFIGURATION VARIABLES
# ============================================================================
# These variables are used across all sections for consistent evaluation

# Verify required variables exist before setting globals
if 'data' not in globals() or 'target_column' not in globals():
    raise ValueError("❌ ERROR: 'data' and 'target_column' must be defined before setting global variables. Please run the data loading cell first.")

# Set up global variables for use in all model evaluations
TARGET_COLUMN = target_column  # Use the target column from data loading

# 🔧 UPDATED: Preserve dataset-specific RESULTS_DIR that was set in CHUNK_005
# Don't override it with a generic path - maintain the structured approach
if 'RESULTS_DIR' not in globals() or RESULTS_DIR is None:
    # Fallback: reconstruct proper results directory structure
    RESULTS_DIR = f"results/{setup.DATASET_IDENTIFIER}/"
    print(f"⚠️  RESULTS_DIR was missing - using fallback: {RESULTS_DIR}")
else:
    print(f"✅ Using existing RESULTS_DIR: {RESULTS_DIR}")

data = data.copy()    # Create a copy of original data for evaluation functions

# Define categorical columns for all models
categorical_columns = data.select_dtypes(include=['object']).columns.tolist()
if TARGET_COLUMN in categorical_columns:
    categorical_columns.remove(TARGET_COLUMN)  # Remove target from categorical list

# Apply user-specified categorical columns if provided
if 'CATEGORICAL_COLUMNS' in globals() and CATEGORICAL_COLUMNS:
    categorical_columns = CATEGORICAL_COLUMNS
    print(f"   • Using user-specified categorical columns: {categorical_columns}")
else:
    print(f"   • Auto-detected categorical columns: {categorical_columns}")

print("🔧 Global Configuration Summary:")
print(f"   • TARGET_COLUMN: {TARGET_COLUMN}")
print(f"   • RESULTS_DIR: {RESULTS_DIR}")
print(f"   • data shape: {data.shape}")
print(f"   • categorical_columns: {categorical_columns}")

# Create base results directory if it doesn't exist
import os
if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR, exist_ok=True)
    print(f"   • Created base results directory: {RESULTS_DIR}")
else:
    print(f"   • Base results directory already exists: {RESULTS_DIR}")

# Validate that all required variables are now available
required_vars = ['TARGET_COLUMN', 'RESULTS_DIR', 'data', 'categorical_columns']
missing_vars = [var for var in required_vars if var not in globals()]

if missing_vars:
    raise ValueError(f"❌ ERROR: Missing required global variables: {missing_vars}")
else:
    print("✅ All required global variables are now available for Section 3 evaluations")

✅ Using existing RESULTS_DIR: results/pakistani-diabetes-dataset/
   • Using user-specified categorical columns: ['Gender', 'Rgn']
🔧 Global Configuration Summary:
   • TARGET_COLUMN: Outcome
   • RESULTS_DIR: results/pakistani-diabetes-dataset/
   • data shape: (500, 19)
   • categorical_columns: ['Gender', 'Rgn']
   • Base results directory already exists: results/pakistani-diabetes-dataset/
✅ All required global variables are now available for Section 3 evaluations


## 3 Demo All Models with Default Parameters

### 3.1 Demos

Each model is run with default parameters for purposes of testing.

#### 3.1.1 CTGAN Demo

In [13]:
# Code Chunk ID: CHUNK_3_1_1_A
import time
try:
    print("🔄 CTGAN Demo - Default Parameters")
    print("=" * 500)
    
    # Import and initialize CTGAN model using ModelFactory
    from src.models.model_factory import ModelFactory
    
    ctgan_model = ModelFactory.create("ctgan", random_state=42)
    
    # Define demo parameters for quick execution
    demo_params = {
        'epochs': 500,
        'batch_size': 100,
        'generator_dim': (128, 128),
        'discriminator_dim': (128, 128)
    }
    
    # Train with demo parameters
    print("Training CTGAN with demo parameters...")
    start_time = time.time()
    
    # Auto-detect discrete columns
    discrete_columns = data.select_dtypes(include=['object']).columns.tolist()
    
    ctgan_model.train(data, discrete_columns=discrete_columns, **demo_params)
    train_time = time.time() - start_time
    
    # Generate synthetic data
    demo_samples = len(data)  # Same size as original dataset
    print(f"Generating {demo_samples} synthetic samples...")
    synthetic_data_ctgan = ctgan_model.generate(demo_samples)
    
    print(f"✅ CTGAN Demo completed successfully!")
    print(f"   - Training time: {train_time:.2f} seconds")
    print(f"   - Generated samples: {len(synthetic_data_ctgan)}")
    print(f"   - Original data shape: {data.shape}")
    print(f"   - Synthetic data shape: {synthetic_data_ctgan.shape}")
    
    # Store for later use in comprehensive evaluation
    demo_results_ctgan = {
        'model': ctgan_model,
        'synthetic_data': synthetic_data_ctgan,
        'training_time': train_time,
        'parameters_used': demo_params
    }
    
except ImportError as e:
    print(f"❌ CTGAN not available: {e}")
    print(f"   Please ensure CTGAN dependencies are installed")
except Exception as e:
    print(f"❌ Error during CTGAN demo: {str(e)}")
    print("   Check model implementation and data compatibility")
    import traceback
    traceback.print_exc()

🔄 CTGAN Demo - Default Parameters
Training CTGAN with demo parameters...


Gen. (-1.72) | Discrim. (-0.23): 100%|██████████| 500/500 [00:19<00:00, 25.79it/s]


Generating 500 synthetic samples...
✅ CTGAN Demo completed successfully!
   - Training time: 28.22 seconds
   - Generated samples: 500
   - Original data shape: (500, 19)
   - Synthetic data shape: (500, 19)


#### 3.1.2 CTAB-GAN Demo

In [14]:
# Code Chunk ID: CHUNK_3_1_2_A
try:
    print("🔄 CTAB-GAN Demo - Default Parameters")
    print("=" * 50)
    
    # Check CTABGAN availability (imported from setup.py)
    if not CTABGAN_AVAILABLE:
        raise ImportError("CTAB-GAN not available - clone and install CTAB-GAN repository")
    
    # Initialize CTAB-GAN model (already defined in notebook)
    ctabgan_model = CTABGANModel()
    print("✅ CTAB-GAN model initialized successfully")
    
    # Record start time
    start_time = time.time()
    
    # Train the model with demo parameters
    print("🚀 Training CTAB-GAN model (epochs=500)...")
    ctabgan_model.fit(data, categorical_columns=None, target_column=target_column)
    
    # Record training time
    train_time = time.time() - start_time
    
    # Generate synthetic data
    print("🎯 Generating synthetic data...")
    synthetic_data_ctabgan = ctabgan_model.generate(len(data))
    
    # Display results
    print("✅ CTAB-GAN Demo completed successfully!")
    print(f"   - Training time: {train_time:.2f} seconds")
    print(f"   - Generated samples: {len(synthetic_data_ctabgan)}")
    print(f"   - Original shape: {data.shape}")
    print(f"   - Synthetic shape: {synthetic_data_ctabgan.shape}")
    
    # Show sample of synthetic data with proper handling for both DataFrame and array
    print(f"\n📊 Sample of generated data:")
    if hasattr(synthetic_data_ctabgan, 'head'):
        # It's a DataFrame
        print(synthetic_data_ctabgan.head())
    else:
        # It's likely a numpy array
        print("First 5 rows of synthetic data:")
        print(synthetic_data_ctabgan[:5])
    print("=" * 50)
    
except ImportError as e:
    print(f"❌ CTAB-GAN not available: {e}")
    print(f"   Please ensure CTAB-GAN dependencies are installed")
    print(f"   Note: CTABGAN_AVAILABLE = {globals().get('CTABGAN_AVAILABLE', 'undefined')}")
except Exception as e:
    print(f"❌ Error during CTAB-GAN demo: {str(e)}")
    print("   Check model implementation and data compatibility")
    import traceback
    traceback.print_exc()

🔄 CTAB-GAN Demo - Default Parameters
✅ CTAB-GAN model initialized successfully
🚀 Training CTAB-GAN model (epochs=500)...
[CTABGAN] Applying comprehensive data preprocessing...
[DATA_CLEANING] Processing 500 rows, 19 columns
[DATA_CLEANING] Categorical columns: []
[DATA_CLEANING] Data cleaning completed successfully
[DATA_CLEANING] Final shape: (500, 19)
[DATA_CLEANING] Data types: {'Age': dtype('float64'), 'Gender': dtype('int64'), 'Rgn ': dtype('int64'), 'wt': dtype('float64'), 'BMI': dtype('float64'), 'wst': dtype('float64'), 'sys': dtype('int64'), 'dia': dtype('int64'), 'his': dtype('int64'), 'A1c': dtype('float64'), 'B.S.R': dtype('int64'), 'vision': dtype('int64'), 'Exr': dtype('int64'), 'dipsia': dtype('int64'), 'uria': dtype('int64'), 'Dur': dtype('float64'), 'neph': dtype('int64'), 'HDL': dtype('int64'), 'Outcome': dtype('int64')}
[CTABGAN] Using categorical columns: []
[CTABGAN] Data shape after preprocessing: (500, 19)
[CTABGAN] Training CTAB-GAN for 100 epochs...


100%|██████████| 100/100 [00:56<00:00,  1.78it/s]

[OK] CTAB-GAN training completed successfully
🎯 Generating synthetic data...
[CTABGAN] Generated 500 raw synthetic samples
[OK] CTAB-GAN generation completed: (500, 19)
✅ CTAB-GAN Demo completed successfully!
   - Training time: 57.97 seconds
   - Generated samples: 500
   - Original shape: (500, 19)
   - Synthetic shape: (500, 19)

📊 Sample of generated data:
         Age    Gender      Rgn          wt        BMI        wst         sys  \
0  40.572305  1.000474  0.967940  73.958330  22.228732  38.100712  142.313983   
1  19.977816  1.015064 -0.001998  63.142142  23.135474  38.104845  139.848295   
2  19.971097  0.002555 -0.015933  68.160717  25.464900  30.802088  111.268934   
3  39.341226  1.000264 -0.004033  76.934273  22.716181  33.888869  138.561703   
4  20.495254  0.992156 -0.002925  60.916312  26.034070  37.280302  105.544096   

         dia       his        A1c       B.S.R    vision        Exr    dipsia  \
0  96.976033  1.007857   7.589860   75.836545 -0.011040  -1.180292  0.

#### 3.1.3 CTAB-GAN+ Demo

In [15]:
# Code Chunk ID: CHUNK_3_1_3_A
try:
    print("🔄 CTAB-GAN+ Demo - Default Parameters")
    print("=" * 50)
    
    # Check CTABGAN+ availability with fallback
    try:
        ctabganplus_available = CTABGANPLUS_AVAILABLE
    except NameError:
        print("⚠️  CTABGANPLUS_AVAILABLE variable not defined - checking direct import...")
        try:
            # Try to check if CTABGANPLUS (the imported class) exists
            from model.ctabgan import CTABGAN as CTABGANPLUS
            ctabganplus_available = True
            print("✅ CTAB-GAN+ import check successful")
        except ImportError:
            ctabganplus_available = False
            print("❌ CTAB-GAN+ import check failed")
    
    if not ctabganplus_available:
        raise ImportError("CTAB-GAN+ not available - clone and install CTAB-GAN+ repository")
    
    # Initialize CTAB-GAN+ model with epochs parameter in constructor
    ctabganplus_model = CTABGANPlusModel(epochs=500)
    print("✅ CTAB-GAN+ model initialized successfully")
    
    # Record start time
    start_time = time.time()
    
    # Train the model (epochs already set in constructor)
    print("🚀 Training CTAB-GAN+ model (epochs=500)...")
    ctabganplus_model.fit(data)
    
    # Record training time
    train_time = time.time() - start_time
    
    # Generate synthetic data
    print("🎯 Generating synthetic data...")
    synthetic_data_ctabganplus = ctabganplus_model.generate(len(data))
    
    # Display results
    print("✅ CTAB-GAN+ Demo completed successfully!")
    print(f"   - Training time: {train_time:.2f} seconds")
    print(f"   - Generated samples: {len(synthetic_data_ctabganplus)}")
    print(f"   - Original shape: {data.shape}")
    print(f"   - Synthetic shape: {synthetic_data_ctabganplus.shape}")
    
    # Show sample of synthetic data with proper handling for both DataFrame and array
    print(f"\n📊 Sample of generated data:")
    if hasattr(synthetic_data_ctabganplus, 'head'):
        # It's a DataFrame
        print(synthetic_data_ctabganplus.head())
    else:
        # It's likely a numpy array
        print("First 5 rows of synthetic data:")
        print(synthetic_data_ctabganplus[:5])
    print("=" * 50)
    
except ImportError as e:
    print(f"❌ CTAB-GAN+ not available: {e}")
    print(f"   Please ensure CTAB-GAN+ dependencies are installed")
except Exception as e:
    print(f"❌ Error during CTAB-GAN+ demo: {str(e)}")
    print("   Check model implementation and data compatibility")
    import traceback
    traceback.print_exc()

🔄 CTAB-GAN+ Demo - Default Parameters
✅ CTAB-GAN+ model initialized successfully
🚀 Training CTAB-GAN+ model (epochs=500)...
[CTABGAN+] Applying comprehensive data preprocessing...
[DATA_CLEANING] Processing 500 rows, 19 columns
[DATA_CLEANING] Categorical columns: []
[DATA_CLEANING] Data cleaning completed successfully
[DATA_CLEANING] Final shape: (500, 19)
[DATA_CLEANING] Data types: {'Age': dtype('float64'), 'Gender': dtype('int64'), 'Rgn ': dtype('int64'), 'wt': dtype('float64'), 'BMI': dtype('float64'), 'wst': dtype('float64'), 'sys': dtype('int64'), 'dia': dtype('int64'), 'his': dtype('int64'), 'A1c': dtype('float64'), 'B.S.R': dtype('int64'), 'vision': dtype('int64'), 'Exr': dtype('int64'), 'dipsia': dtype('int64'), 'uria': dtype('int64'), 'Dur': dtype('float64'), 'neph': dtype('int64'), 'HDL': dtype('int64'), 'Outcome': dtype('int64')}
[CTABGAN+] Using categorical columns: []
[CTABGAN+] Data shape after preprocessing: (500, 19)
[CTABGAN+] Using Classification with target: Outcom

100%|██████████| 1/1 [00:00<00:00,  1.70it/s]

Finished training in 2.1946346759796143  seconds.
[OK] CTAB-GAN+ training completed successfully
🎯 Generating synthetic data...
[CTABGAN+] Generated 500 raw synthetic samples
[OK] CTAB-GAN+ generation completed: (500, 19)
✅ CTAB-GAN+ Demo completed successfully!
   - Training time: 2.23 seconds
   - Generated samples: 500
   - Original shape: (500, 19)
   - Synthetic shape: (500, 19)

📊 Sample of generated data:
         Age  Gender  Rgn          wt        BMI        wst  sys  dia  his  \
0  54.106183       1     1  72.363135  26.245544  35.224419  186   71    0   
1  24.199008       1     0  72.481016  26.264188  35.209709  126   74    0   
2  24.221342       1     0  84.546232  30.614124  40.446934  126   71    1   
3  54.093227       1     1  84.331606  30.635607  35.204397  185   74    1   
4  24.212883       0     0  72.615425  37.366556  35.226785  139   71    0   

        A1c  B.S.R  vision  Exr  dipsia  uria        Dur  neph  HDL  Outcome  
0  5.450221    217       1  102     

#### 3.1.4 GANerAid Demo

In [16]:
# Code Chunk ID: CHUNK_3_1_4_A
try:
    print("🔄 GANerAid Demo - Default Parameters")
    print("=" * 50)
    
    # Check GANerAid availability with fallback
    try:
        ganeraid_available = GANERAID_AVAILABLE
        GANerAidModel  # Test if the class is available
    except NameError:
        print("⚠️ GANerAidModel not available - checking import...")
        try:
            # Try to import GANerAidModel
            from src.models.implementations.ganeraid_model import GANerAidModel
            ganeraid_available = True
            print("✅ GANerAidModel import successful")
        except ImportError:
            ganeraid_available = False
            print("❌ GANerAidModel import failed")
    
    if not ganeraid_available:
        raise ImportError("GANerAid not available - please install GANerAid dependencies")
    
    # Initialize GANerAid model
    ganeraid_model = GANerAidModel()
    print("✅ GANerAid model initialized successfully")
    
    # Define demo_samples variable for synthetic data generation
    demo_samples = len(data)  # Same size as original dataset
    
    # Train with minimal parameters for demo
    demo_params = {'epochs': 500, 'batch_size': 100}
    start_time = time.time()
    ganeraid_model.train(data, **demo_params)  # GANerAid uses train method
    train_time = time.time() - start_time
    
    # Generate synthetic data
    synthetic_data_ganeraid = ganeraid_model.generate(demo_samples)
    
    print(f"✅ GANerAid Demo completed successfully!")
    print(f"   - Training time: {train_time:.2f} seconds")
    print(f"   - Generated samples: {len(synthetic_data_ganeraid)}")
    print(f"   - Original shape: {data.shape}")
    print(f"   - Synthetic shape: {synthetic_data_ganeraid.shape}")
    
    # Show sample of synthetic data with proper handling for both DataFrame and array
    print(f"\n📊 Sample of generated data:")
    if hasattr(synthetic_data_ganeraid, 'head'):
        # It's a DataFrame
        print(synthetic_data_ganeraid.head())
    else:
        # It's likely a numpy array
        print("First 5 rows of synthetic data:")
        print(synthetic_data_ganeraid[:5])
    print("=" * 50)
    
except ImportError as e:
    print(f"❌ GANerAid not available: {e}")
    print(f"   Please ensure GANerAid dependencies are installed")
except Exception as e:
    print(f"❌ Error during GANerAid demo: {str(e)}")
    print("   Check model implementation and data compatibility")
    import traceback
    traceback.print_exc()

🔄 GANerAid Demo - Default Parameters
✅ GANerAid model initialized successfully
Initialized gan with the following parameters: 
lr_d = 0.0005
lr_g = 0.0005
hidden_feature_space = 200
batch_size = 100
nr_of_rows = 25
binary_noise = 0.2
Start training of gan for 500 epochs


100%|██████████| 500/500 [00:30<00:00, 16.60it/s, loss=d error: 0.37797756120562553 --- g error 5.518411636352539]  


Generating 500 samples
✅ GANerAid Demo completed successfully!
   - Training time: 30.24 seconds
   - Generated samples: 500
   - Original shape: (500, 19)
   - Synthetic shape: (500, 19)

📊 Sample of generated data:
         Age  Gender  Rgn          wt        BMI        wst  sys  dia  his  \
0  40.121815       1     0  66.824463  28.168720  35.622356  149   74    1   
1  38.745453       0     0  66.637581  26.680719  34.941753  149   76    0   
2  37.212540       0     0  66.845291  27.762980  35.659401  133   80    0   
3  40.797607       0     0  63.893791  27.088366  34.319633  125   82    1   
4  38.781696       1     0  62.305569  26.830305  33.214291  141   74    1   

        A1c  B.S.R  vision  Exr  dipsia  uria       Dur  neph  HDL  Outcome  
0  6.254515    249       0   40       1     0  6.383238     0   44        1  
1  6.946755    256       0   42       0     0  8.605595     0   45        1  
2  7.126358    229       0   24       0     0  3.526692     0   45        1  
3 

#### 3.1.5 CopulaGAN Demo

In [17]:
# Code Chunk ID: CHUNK_3_1_5_A
try:
    print("🔄 CopulaGAN Demo - Default Parameters")
    print("=" * 50)
    
    # Import and initialize CopulaGAN model using ModelFactory
    from src.models.model_factory import ModelFactory
    
    copulagan_model = ModelFactory.create("copulagan", random_state=42)
    
    # Define demo parameters optimized for CopulaGAN
    demo_params = {
        'epochs': 500,
        'batch_size': 100,
        'generator_dim': (128, 128),
        'discriminator_dim': (128, 128),
        'default_distribution': 'beta',  # Good for bounded data
        'enforce_min_max_values': True
    }
    
    # Train with demo parameters
    print("Training CopulaGAN with demo parameters...")
    start_time = time.time()
    
    # Auto-detect discrete columns for CopulaGAN
    discrete_columns = data.select_dtypes(include=['object']).columns.tolist()
    
    copulagan_model.train(data, discrete_columns=discrete_columns, **demo_params)
    train_time = time.time() - start_time
    
    # Generate synthetic data
    demo_samples = len(data)  # Same size as original dataset
    print(f"Generating {demo_samples} synthetic samples...")
    synthetic_data_copulagan = copulagan_model.generate(demo_samples)
    
    print(f"✅ CopulaGAN Demo completed successfully!")
    print(f"   - Training time: {train_time:.2f} seconds")
    print(f"   - Generated samples: {len(synthetic_data_copulagan)}")
    print(f"   - Original data shape: {data.shape}")
    print(f"   - Synthetic data shape: {synthetic_data_copulagan.shape}")
    print(f"   - Distribution used: {demo_params['default_distribution']}")
    
    # Store for later use in comprehensive evaluation
    demo_results_copulagan = {
        'model': copulagan_model,
        'synthetic_data': synthetic_data_copulagan,
        'training_time': train_time,
        'parameters_used': demo_params
    }
    
except ImportError as e:
    print(f"❌ CopulaGAN not available: {e}")
    print(f"   Please ensure CopulaGAN dependencies are installed")
except Exception as e:
    print(f"❌ Error during CopulaGAN demo: {str(e)}")
    print("   Check model implementation and data compatibility")
    import traceback
    traceback.print_exc()

🔄 CopulaGAN Demo - Default Parameters
Training CopulaGAN with demo parameters...
Generating 500 synthetic samples...
✅ CopulaGAN Demo completed successfully!
   - Training time: 62.25 seconds
   - Generated samples: 500
   - Original data shape: (500, 19)
   - Synthetic data shape: (500, 19)
   - Distribution used: beta


#### 3.1.6 TVAE Demo

In [18]:
# Code Chunk ID: CHUNK_3_1_6_A
try:
    print("🔄 TVAE Demo - Default Parameters")
    print("=" * 50)
    
    # Import and initialize TVAE model using ModelFactory
    from src.models.model_factory import ModelFactory
    
    tvae_model = ModelFactory.create("tvae", random_state=42)
    
    # Define demo parameters optimized for TVAE
    demo_params = {
        'epochs': 50,
        'batch_size': 100,
        'compress_dims': (128, 128),
        'decompress_dims': (128, 128),
        'l2scale': 1e-5,
        'loss_factor': 2,
        'learning_rate': 1e-3  # VAE-specific learning rate
    }
    
    # Train with demo parameters
    print("Training TVAE with demo parameters...")
    start_time = time.time()
    
    # Auto-detect discrete columns for TVAE
    discrete_columns = data.select_dtypes(include=['object']).columns.tolist()
    
    tvae_model.train(data, discrete_columns=discrete_columns, **demo_params)
    train_time = time.time() - start_time
    
    # Generate synthetic data
    demo_samples = len(data)  # Same size as original dataset
    print(f"Generating {demo_samples} synthetic samples...")
    synthetic_data_tvae = tvae_model.generate(demo_samples)
    
    print(f"✅ TVAE Demo completed successfully!")
    print(f"   - Training time: {train_time:.2f} seconds")
    print(f"   - Generated samples: {len(synthetic_data_tvae)}")
    print(f"   - Original data shape: {data.shape}")
    print(f"   - Synthetic data shape: {synthetic_data_tvae.shape}")
    print(f"   - VAE architecture: compress{demo_params['compress_dims']} → decompress{demo_params['decompress_dims']}")
    
    # Store for later use in comprehensive evaluation
    demo_results_tvae = {
        'model': tvae_model,
        'synthetic_data': synthetic_data_tvae,
        'training_time': train_time,
        'parameters_used': demo_params
    }
    
except ImportError as e:
    print(f"❌ TVAE not available: {e}")
    print(f"   Please ensure TVAE dependencies are installed")
except Exception as e:
    print(f"❌ Error during TVAE demo: {str(e)}")
    print("   Check model implementation and data compatibility")
    import traceback
    traceback.print_exc()

🔄 TVAE Demo - Default Parameters
Training TVAE with demo parameters...
Generating 500 synthetic samples...
✅ TVAE Demo completed successfully!
   - Training time: 2.66 seconds
   - Generated samples: 500
   - Original data shape: (500, 19)
   - Synthetic data shape: (500, 19)
   - VAE architecture: compress(128, 128) → decompress(128, 128)


### 3.2 Batch Process

This section outputs figures and graphics from models run in 3.1. The next chunk will output results for whatever subsections of 3.1 were run. I.e., if there's need to debug one method, there's no need to run all subsections of 3.1.

In [19]:
# Code Chunk ID: CHUNK_3_2_0_A
# ============================================================================
# SECTION 3 - BATCH EVALUATION FOR ALL TRAINED MODELS
# Standardized evaluation using enhanced batch evaluation system
# ============================================================================

print("🔍 SECTION 3 - COMPREHENSIVE BATCH EVALUATION")
print("=" * 60)

section3_results = evaluate_all_available_models(
    section_number=3,
    scope=globals(),  # Pass notebook scope to access synthetic data variables
    models_to_evaluate=None,  # Evaluate all available models
    real_data=None,  # Will use 'data' from scope
    target_col=None   # Will use 'target_column' from scope
)

if section3_results:
    print(f"\n🎉 SECTION 3 BATCH EVALUATION COMPLETED!")
    print(f"📊 Evaluated {len(section3_results)} models successfully")
    print(f"📁 All results saved to organized folder structure")
    
    # Show quick summary of best performing models
    best_models = []
    for model_name, results in section3_results.items():
        if 'error' not in results:
            quality_score = results.get('overall_quality_score', 0)
            best_models.append((model_name, quality_score))
    
    if best_models:
        best_models.sort(key=lambda x: x[1], reverse=True)
        print(f"\n🏆 RANKING BY QUALITY SCORE:")
        for i, (model, score) in enumerate(best_models, 1):
            print(f"   {i}. {model}: {score:.3f}")
else:
    print("\n⚠️ No models available for evaluation")
    print("   Train some models first in previous sections")

🔍 SECTION 3 - COMPREHENSIVE BATCH EVALUATION
[SEARCH] UNIFIED BATCH EVALUATION - SECTION 3
[INFO] Dataset: pakistani-diabetes-dataset
[INFO] Target column: Outcome
[INFO] Variable pattern: standard
[INFO] Found 6 trained models:
   [OK] CTGAN
   [OK] CTABGAN
   [OK] CTABGANPLUS
   [OK] GANerAid
   [OK] CopulaGAN
   [OK] TVAE

==================== EVALUATING CTGAN ====================
[SEARCH] CTGAN - COMPREHENSIVE DATA QUALITY EVALUATION
[FOLDER] Output directory: results\pakistani-diabetes-dataset\2025-09-23\Section-3\CTGAN

[1] STATISTICAL SIMILARITY
------------------------------
   [CHART] Average Statistical Similarity: 0.768

[2] PCA COMPARISON ANALYSIS WITH OUTCOME COLOR-CODING
--------------------------------------------------
   [CHART] PCA comparison plot saved: pca_comparison_with_outcome.png
   [CHART] PCA Overall Similarity: 0.031
   [CHART] Explained Variance (PC1, PC2): 0.312, 0.092

[3] DISTRIBUTION SIMILARITY
------------------------------
   [CHART] Average Distributi

## 4: Hyperparameter Tuning for Each Model

### 4.1 Hyperparameter Optimization

#### 4.1.1 CTGAN Hyperparameter Optimization

In [20]:
# Code Chunk ID: CHUNK_4_1_1_A
# CTGAN Hyperparameter Optimization Execution
# Complete optimization study with search space definition and execution

# Import required libraries
import optuna
import numpy as np
import pandas as pd
from scipy.stats import wasserstein_distance  # FIXED: Add missing wasserstein_distance import

def ctgan_search_space(trial):
    """Define CTGAN hyperparameter search space with corrected PAC validation."""
    # Select batch size first
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256, 500, 1000])
    
    # PAC must be <= batch_size and batch_size must be divisible by PAC
    max_pac = min(20, batch_size)
    pac = trial.suggest_int('pac', 1, max_pac)
    
    return {
        'epochs': trial.suggest_int('epochs', 100, 1000, step=50),
        'batch_size': batch_size,
        'generator_lr': trial.suggest_loguniform('generator_lr', 5e-6, 5e-3),
        'discriminator_lr': trial.suggest_loguniform('discriminator_lr', 5e-6, 5e-3),
        'generator_dim': trial.suggest_categorical('generator_dim', [
            (128, 128),
            (256, 256), 
            (512, 256),
            (256, 512),
            (512, 512),
            (128, 256, 128),
            (256, 128, 64),
            (256, 512, 256)
        ]),
        'discriminator_dim': trial.suggest_categorical('discriminator_dim', [
            (128, 128),
            (256, 256),
            (256, 512), 
            (512, 256),
            (128, 256, 128),
            (256, 512, 256)
        ]),
        'pac': pac,
        'discriminator_steps': trial.suggest_int('discriminator_steps', 1, 5),
        'generator_decay': trial.suggest_loguniform('generator_decay', 1e-8, 1e-4),
        'discriminator_decay': trial.suggest_loguniform('discriminator_decay', 1e-8, 1e-4),
        'log_frequency': trial.suggest_categorical('log_frequency', [True, False]),
        'verbose': trial.suggest_categorical('verbose', [True])
    }

def ctgan_objective(trial):
    """CTGAN objective function with corrected PAC validation and fixed imports."""
    try:
        # Get hyperparameters from trial
        params = ctgan_search_space(trial)
        
        # CORRECTED PAC VALIDATION: Fix incompatible combinations if needed
        batch_size = params['batch_size']
        original_pac = params['pac']
        
        # Find the largest compatible PAC value <= original_pac
        compatible_pac = original_pac
        while compatible_pac > 1 and batch_size % compatible_pac != 0:
            compatible_pac -= 1
        
        # Update PAC to be compatible
        if compatible_pac != original_pac:
            print(f"🔧 PAC adjusted: {original_pac} → {compatible_pac} (for batch_size={batch_size})")
            params['pac'] = compatible_pac
        
        print(f"\n🔄 CTGAN Trial {trial.number + 1}: epochs={params['epochs']}, batch_size={params['batch_size']}, pac={params['pac']}, lr={params['generator_lr']:.2e}")
        print(f"✅ PAC validation: {params['batch_size']} % {params['pac']} = {params['batch_size'] % params['pac']}")
        
        # FIXED: Use proper TARGET_COLUMN from global scope
        global TARGET_COLUMN
        if 'TARGET_COLUMN' not in globals() or TARGET_COLUMN is None:
            TARGET_COLUMN = data.columns[-1]  # Use last column as fallback
        print(f"🎯 Using target column: '{TARGET_COLUMN}'")
        
        # FIXED: Use correct CTGAN import - try multiple import paths
        try:
            from ctgan import CTGAN
            print("✅ Using CTGAN from ctgan package")
        except ImportError:
            try:
                from sdv.single_table import CTGANSynthesizer
                CTGAN = CTGANSynthesizer
                print("✅ Using CTGANSynthesizer from sdv.single_table")
            except ImportError:
                try:
                    from sdv.tabular import CTGAN
                    print("✅ Using CTGAN from sdv.tabular")
                except ImportError:
                    raise ImportError("❌ Could not import CTGAN from any known package")
        
        # Auto-detect discrete columns  
        if 'data' not in globals():
            raise ValueError("Data not available in global scope")
            
        discrete_columns = data.select_dtypes(include=['object']).columns.tolist()
        
        # FIXED: Initialize CTGAN using flexible approach
        try:
            # Try direct CTGAN initialization
            model = CTGAN(
                epochs=params['epochs'],
                batch_size=params['batch_size'],
                generator_lr=params['generator_lr'],
                discriminator_lr=params['discriminator_lr'],
                generator_dim=params['generator_dim'],
                discriminator_dim=params['discriminator_dim'],
                pac=params['pac'],
                discriminator_steps=params['discriminator_steps'],
                generator_decay=params['generator_decay'],
                discriminator_decay=params['discriminator_decay'],
                log_frequency=params['log_frequency'],
                verbose=params['verbose']
            )
        except TypeError as te:
            # Fallback: Use basic parameters only
            print(f"⚠️ Full parameter initialization failed, using basic parameters: {te}")
            model = CTGAN(
                epochs=params['epochs'],
                batch_size=params['batch_size'],
                pac=params['pac'],
                verbose=params['verbose']
            )
        
        # Train the model
        model.fit(data)
        
        # Generate synthetic data
        synthetic_data = model.sample(len(data))
        
        # Use enhanced objective function with proper target column passing
        score, similarity_score, accuracy_score = enhanced_objective_function_v2(
            data, synthetic_data, TARGET_COLUMN
        )
        
        print(f"✅ CTGAN Trial {trial.number + 1} Score: {score:.4f} (Similarity: {similarity_score:.4f}, Accuracy: {accuracy_score:.4f})")
        
        return score
        
    except Exception as e:
        print(f"❌ CTGAN trial {trial.number + 1} failed: {str(e)}")
        print(f"🔍 Error details: {type(e).__name__}(\"{str(e)}\")") 
        import traceback
        traceback.print_exc()
        return 0.0

print("🎯 Starting CTGAN Hyperparameter Optimization - FIXED IMPORT")
print(f"   • Target column: '{TARGET_COLUMN}' (dynamic detection)")

# Create the optimization study
ctgan_study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=2)
)

# Run the optimization
try:
    # Ensure we have the required global variables
    if 'data' not in globals():
        raise ValueError("❌ Data not available - please run data loading sections first")
        
    if 'TARGET_COLUMN' not in globals():
        TARGET_COLUMN = data.columns[-1]  # Use last column as fallback
        print(f"⚠️  TARGET_COLUMN not set, using fallback: '{TARGET_COLUMN}'")
    
    print(f"📊 Dataset info: {data.shape[0]} rows, {data.shape[1]} columns")
    print(f"📊 Target column '{TARGET_COLUMN}' unique values: {data[TARGET_COLUMN].nunique()}")
    print()
    
    # Run the optimization trials
    ctgan_study.optimize(ctgan_objective, n_trials=5)  
    
    # Display results
    print(f"\n📊 CTGAN hyperparameter optimization with corrected PAC compatibility completed!")
    print("🎯 No more dynamic parameter name issues - simplified and robust approach")
    
    # Best trial information
    best_trial = ctgan_study.best_trial
    print(f"\n🏆 Best Trial Results:")
    print(f"   • Best Score: {best_trial.value:.4f}")
    print(f"   • Trial Number: {best_trial.number}")
    print(f"   • Best Parameters:")
    for key, value in best_trial.params.items():
        print(f"     - {key}: {value}")
    
    # Summary statistics
    completed_trials = [t for t in ctgan_study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    failed_trials = [t for t in ctgan_study.trials if t.state == optuna.trial.TrialState.FAIL]
    
    print(f"\n📈 Optimization Summary:")
    print(f"   • Total trials completed: {len(completed_trials)}")
    print(f"   • Failed trials: {len(failed_trials)}")
    if completed_trials:
        scores = [t.value for t in completed_trials]
        print(f"   • Best score: {max(scores):.4f}")
        print(f"   • Mean score: {sum(scores)/len(scores):.4f}")
        print(f"   • Score range: {max(scores) - min(scores):.4f}")
    
    # Store results for Section 4.1.1 analysis
    ctgan_optimization_results = {
        'study': ctgan_study,
        'best_trial': best_trial,
        'completed_trials': completed_trials,
        'failed_trials': failed_trials
    }
    
    print(f"\n✅ CTGAN optimization data ready for Section 4.1.1 analysis")
    
except Exception as e:
    print(f"❌ CTGAN hyperparameter optimization failed: {str(e)}")
    print(f"🔍 Error details: {repr(e)}")
    import traceback
    traceback.print_exc()
    
    # Create dummy results for analysis to continue
    ctgan_study = None
    ctgan_optimization_results = None
    print("⚠️  CTGAN optimization failed - Section 4.1.1 analysis will show 'data not found' message")

[I 2025-09-23 17:04:09,330] A new study created in memory with name: no-name-c6a9112a-42dc-4ecf-aaf4-59fd028232e9


🎯 Starting CTGAN Hyperparameter Optimization - FIXED IMPORT
   • Target column: 'Outcome' (dynamic detection)
📊 Dataset info: 500 rows, 19 columns
📊 Target column 'Outcome' unique values: 2

🔧 PAC adjusted: 13 → 10 (for batch_size=1000)

🔄 CTGAN Trial 1: epochs=450, batch_size=1000, pac=10, lr=3.13e-03
✅ PAC validation: 1000 % 10 = 0
🎯 Using target column: 'Outcome'
✅ Using CTGAN from ctgan package


Gen. (-1.18) | Discrim. (0.02): 100%|██████████| 450/450 [01:22<00:00,  5.48it/s] 
[I 2025-09-23 17:05:33,611] Trial 0 finished with value: 0.39404558152069885 and parameters: {'batch_size': 1000, 'pac': 13, 'epochs': 450, 'generator_lr': 0.003127749065256762, 'discriminator_lr': 7.47016108443751e-06, 'generator_dim': (256, 512, 256), 'discriminator_dim': (128, 128), 'discriminator_steps': 3, 'generator_decay': 1.2544121396819714e-07, 'discriminator_decay': 1.492349675590579e-06, 'log_frequency': True, 'verbose': True}. Best is trial 0 with value: 0.39404558152069885.


[TARGET] Enhanced objective function using target column: 'Outcome'
[OK] Similarity Analysis: 19/19 valid metrics, Average: 0.3694
[OK] TRTS (Synthetic->Real): 0.5300
[OK] TRTS Evaluation: 2 scenarios, Average: 0.4310
[CHART] Combined Score: 0.3940 (Similarity: 0.3694, Accuracy: 0.4310)
✅ CTGAN Trial 1 Score: 0.3940 (Similarity: 0.3694, Accuracy: 0.4310)
🔧 PAC adjusted: 18 → 10 (for batch_size=1000)

🔄 CTGAN Trial 2: epochs=200, batch_size=1000, pac=10, lr=3.04e-05
✅ PAC validation: 1000 % 10 = 0
🎯 Using target column: 'Outcome'
✅ Using CTGAN from ctgan package


Gen. (-0.67) | Discrim. (-0.25): 100%|██████████| 200/200 [00:36<00:00,  5.47it/s]
[I 2025-09-23 17:06:12,303] Trial 1 finished with value: 0.5682618468222216 and parameters: {'batch_size': 1000, 'pac': 18, 'epochs': 200, 'generator_lr': 3.0363161079567684e-05, 'discriminator_lr': 4.9067516856653705e-05, 'generator_dim': (256, 512), 'discriminator_dim': (128, 128), 'discriminator_steps': 5, 'generator_decay': 4.4043327199051804e-08, 'discriminator_decay': 1.2310021859892126e-08, 'log_frequency': False, 'verbose': True}. Best is trial 1 with value: 0.5682618468222216.


...

🔧 PAC adjusted: 13 → 8 (for batch_size=64)

🔄 CTGAN Trial 5: epochs=150, batch_size=64, pac=8, lr=4.55e-05
✅ PAC validation: 64 % 8 = 0
🎯 Using target column: 'Outcome'
✅ Using CTGAN from ctgan package


Gen. (1.49) | Discrim. (-3.08): 100%|██████████| 150/150 [00:40<00:00,  3.71it/s] 


[TARGET] Enhanced objective function using target column: 'Outcome'
[OK] Similarity Analysis: 19/19 valid metrics, Average: 0.6130


[I 2025-09-23 17:09:00,498] Trial 4 finished with value: 0.5378003653455115 and parameters: {'batch_size': 64, 'pac': 13, 'epochs': 150, 'generator_lr': 4.548192666211065e-05, 'discriminator_lr': 0.00038829246570670084, 'generator_dim': (256, 128, 64), 'discriminator_dim': (128, 256, 128), 'discriminator_steps': 2, 'generator_decay': 2.390920102437596e-05, 'discriminator_decay': 7.94830092072193e-06, 'log_frequency': False, 'verbose': True}. Best is trial 2 with value: 0.7256546451390966.


[OK] TRTS (Synthetic->Real): 0.3480
[OK] TRTS Evaluation: 2 scenarios, Average: 0.4250
[CHART] Combined Score: 0.5378 (Similarity: 0.6130, Accuracy: 0.4250)
✅ CTGAN Trial 5 Score: 0.5378 (Similarity: 0.6130, Accuracy: 0.4250)

📊 CTGAN hyperparameter optimization with corrected PAC compatibility completed!
🎯 No more dynamic parameter name issues - simplified and robust approach

🏆 Best Trial Results:
   • Best Score: 0.7257
   • Trial Number: 2
   • Best Parameters:
     - batch_size: 256
     - pac: 2
     - epochs: 450
     - generator_lr: 4.7004557414829124e-05
     - discriminator_lr: 0.002053483212397359
     - generator_dim: (512, 256)
     - discriminator_dim: (128, 128)
     - discriminator_steps: 5
     - generator_decay: 7.693044722780353e-07
     - discriminator_decay: 1.2164482219891981e-06
     - log_frequency: False
     - verbose: True

📈 Optimization Summary:
   • Total trials completed: 5
   • Failed trials: 0
   • Best score: 0.7257
   • Mean score: 0.5278
   • Score r

#### 4.1.2 CTAB-GAN Hyperparameter Optimization

In [21]:
# Code Chunk ID: CHUNK_4_1_2_A
# Import required libraries for CTAB-GAN optimization
import optuna
import numpy as np
import pandas as pd
from src.models.model_factory import ModelFactory
from src.evaluation.trts_framework import TRTSEvaluator

# CORRECTED CTAB-GAN Search Space (3 supported parameters only)
def ctabgan_search_space(trial):
    """Realistic CTAB-GAN hyperparameter space - ONLY supported parameters"""
    return {
        'epochs': trial.suggest_int('epochs', 100, 1000, step=50),
        'batch_size': trial.suggest_categorical('batch_size', [64, 128, 256]),  # Remove 500 - not stable
        'test_ratio': trial.suggest_float('test_ratio', 0.15, 0.25, step=0.05),
        # REMOVED: class_dim, random_dim, num_channels (not supported by constructor)
    }

def ctabgan_objective(trial):
    """FINAL CORRECTED CTAB-GAN objective function with SCORE EXTRACTION FIX"""
    try:
        # Get realistic hyperparameters from trial
        params = ctabgan_search_space(trial)
        
        print(f"\n🔄 CTAB-GAN Trial {trial.number + 1}: epochs={params['epochs']}, batch_size={params['batch_size']}, test_ratio={params['test_ratio']:.3f}")
        
        # Initialize CTAB-GAN using ModelFactory
        model = ModelFactory.create("ctabgan", random_state=42)
        
        # Only pass supported parameters to train()
        result = model.train(data, 
                           epochs=params['epochs'],
                           batch_size=params['batch_size'],
                           test_ratio=params['test_ratio'])
        
        print(f"🏋️ Training CTAB-GAN with corrected parameters...")
        
        # Generate synthetic data for evaluation
        synthetic_data = model.generate(len(data))
        
        # CRITICAL FIX: Convert synthetic data labels to match original data types before TRTS evaluation
        synthetic_data_converted = synthetic_data.copy()
        if target_column in synthetic_data_converted.columns and target_column in data.columns:
            # Convert string labels to numeric to match original data type
            if synthetic_data_converted[target_column].dtype == 'object' and data[target_column].dtype != 'object':
                print(f"🔧 Converting synthetic labels from {synthetic_data_converted[target_column].dtype} to {data[target_column].dtype}")
                synthetic_data_converted[target_column] = pd.to_numeric(synthetic_data_converted[target_column], errors='coerce')
                
                # Handle any conversion failures
                if synthetic_data_converted[target_column].isna().any():
                    print(f"⚠️ Some labels failed conversion - filling with mode")
                    mode_value = data[target_column].mode()[0]
                    synthetic_data_converted[target_column].fillna(mode_value, inplace=True)
                
                # Ensure same data type as original
                synthetic_data_converted[target_column] = synthetic_data_converted[target_column].astype(data[target_column].dtype)
                print(f"✅ Label conversion successful: {synthetic_data_converted[target_column].dtype}")
        
        # Calculate similarity score using TRTS framework with converted data
        trts = TRTSEvaluator(random_state=42)
        trts_results = trts.evaluate_trts_scenarios(data, synthetic_data_converted, target_column=target_column)
        
        # 🎯 CRITICAL FIX: Correct Score Extraction (targeting ML accuracy scores, not percentages)
        if 'trts_scores' in trts_results and isinstance(trts_results['trts_scores'], dict):
            trts_scores = list(trts_results['trts_scores'].values())  # Extract ML accuracy scores (0-1 scale)
            print(f"🎯 CORRECTED: ML accuracy scores = {trts_scores}")
        else:
            # Fallback to filtered method if structure unexpected
            print(f"⚠️ Using fallback score extraction")
            trts_scores = [score for score in trts_results.values() if isinstance(score, (int, float)) and 0 <= score <= 1]
            print(f"🔍 Fallback extracted scores = {trts_scores}")
        
        # CORRECTED EVALUATION FAILURE DETECTION (using proper 0-1 scale)
        if not trts_scores:
            print(f"❌ TRTS evaluation failure: NO NUMERIC SCORES RETURNED")
            return 0.0
        elif all(score >= 0.99 for score in trts_scores):  # Now checking 0-1 scale scores
            print(f"❌ TRTS evaluation failure: ALL SCORES ≥0.99 (suspicious perfect scores)")
            print(f"   • Perfect scores detected: {trts_scores}")
            return 0.0  
        else:
            # TRTS evaluation successful
            similarity_score = np.mean(trts_scores) if trts_scores else 0.0
            similarity_score = max(0.0, min(1.0, similarity_score))
            print(f"✅ TRTS evaluation successful: {similarity_score:.4f} (from {len(trts_scores)} ML accuracy scores)")
        
        # Calculate accuracy with converted labels
        try:
            from sklearn.ensemble import RandomForestClassifier
            from sklearn.metrics import accuracy_score
            from sklearn.model_selection import train_test_split
            
            # Use converted synthetic data for accuracy calculation
            if target_column in data.columns and target_column in synthetic_data_converted.columns:
                X_real = data.drop(target_column, axis=1)
                y_real = data[target_column]
                X_synth = synthetic_data_converted.drop(target_column, axis=1) 
                y_synth = synthetic_data_converted[target_column]
                
                # Train on synthetic, test on real (TRTS approach)
                X_train, X_test, y_train, y_test = train_test_split(X_real, y_real, test_size=0.2, random_state=42)
                
                clf = RandomForestClassifier(random_state=42, n_estimators=50)
                clf.fit(X_synth, y_synth)
                
                predictions = clf.predict(X_test)
                accuracy = accuracy_score(y_test, predictions)
                
                # Combined score (weighted average of similarity and accuracy)
                score = 0.6 * similarity_score + 0.4 * accuracy
                score = max(0.0, min(1.0, score))  # Ensure 0-1 range
                
                print(f"✅ CTAB-GAN Trial {trial.number + 1} Score: {score:.4f} (Similarity: {similarity_score:.4f}, Accuracy: {accuracy:.4f})")
            else:
                score = similarity_score
                print(f"✅ CTAB-GAN Trial {trial.number + 1} Score: {score:.4f} (Similarity: {similarity_score:.4f})")
                
        except Exception as e:
            print(f"⚠️ Accuracy calculation failed: {e}")
            score = similarity_score
            print(f"✅ CTAB-GAN Trial {trial.number + 1} Score: {score:.4f} (Similarity: {similarity_score:.4f})")
        
        return score
        
    except Exception as e:
        print(f"❌ CTAB-GAN trial {trial.number + 1} failed: {str(e)}")
        return 0.0  # FAILED MODELS RETURN 0.0, NOT 1.0

# Execute CTAB-GAN hyperparameter optimization with SCORE EXTRACTION FIX
print("\n🎯 Starting CTAB-GAN Hyperparameter Optimization - SCORE EXTRACTION FIX")
print("   • Search space: 3 supported parameters (epochs, batch_size, test_ratio)")
print("   • Parameter validation: Only constructor-supported parameters")
print("   • 🎯 CRITICAL FIX: Correct ML accuracy score extraction (0-1 scale)")
print("   • Proper threshold detection: Using 0-1 scale for perfect score detection")
print("   • Number of trials: 5")
print(f"   • Algorithm: TPE with median pruning")

# Create and execute study
ctabgan_study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
ctabgan_study.optimize(ctabgan_objective, n_trials=5)

# Display results
print(f"\n✅ CTAB-GAN Optimization with Score Fix Complete:")
print(f"   • Best objective score: {ctabgan_study.best_value:.4f}")
print(f"   • Best hyperparameters:")
for key, value in ctabgan_study.best_params.items():
    if isinstance(value, float):
        print(f"     - {key}: {value:.4f}")
    else:
        print(f"     - {key}: {value}")

# Store best parameters for later use
ctabgan_best_params = ctabgan_study.best_params
print("\n📊 CTAB-GAN hyperparameter optimization with score extraction fix completed!")
print(f"🎯 Expected: Variable scores reflecting actual ML accuracy performance")

[I 2025-09-23 17:09:00,532] A new study created in memory with name: no-name-52785e2a-08ca-4876-8303-0507d659d90f



🎯 Starting CTAB-GAN Hyperparameter Optimization - SCORE EXTRACTION FIX
   • Search space: 3 supported parameters (epochs, batch_size, test_ratio)
   • Parameter validation: Only constructor-supported parameters
   • 🎯 CRITICAL FIX: Correct ML accuracy score extraction (0-1 scale)
   • Proper threshold detection: Using 0-1 scale for perfect score detection
   • Number of trials: 5
   • Algorithm: TPE with median pruning

🔄 CTAB-GAN Trial 1: epochs=700, batch_size=64, test_ratio=0.150


100%|██████████| 700/700 [05:06<00:00,  2.29it/s]


Finished training in 307.656813621521  seconds.
🏋️ Training CTAB-GAN with corrected parameters...
🎯 CORRECTED: ML accuracy scores = [0.9733333333333334, 0.8866666666666667, 0.9133333333333333, 0.9333333333333333]
✅ TRTS evaluation successful: 0.9267 (from 4 ML accuracy scores)


[I 2025-09-23 17:14:08,443] Trial 0 finished with value: 0.9520000000000001 and parameters: {'epochs': 700, 'batch_size': 64, 'test_ratio': 0.15}. Best is trial 0 with value: 0.9520000000000001.


...

✅ CTAB-GAN Trial 4 Score: 0.9520 (Similarity: 0.9267, Accuracy: 0.9900)

🔄 CTAB-GAN Trial 5: epochs=900, batch_size=128, test_ratio=0.200


100%|██████████| 900/900 [06:43<00:00,  2.23it/s]


Finished training in 405.285941362381  seconds.
🏋️ Training CTAB-GAN with corrected parameters...
🎯 CORRECTED: ML accuracy scores = [0.9733333333333334, 0.88, 0.94, 0.8733333333333333]
✅ TRTS evaluation successful: 0.9167 (from 4 ML accuracy scores)


[I 2025-09-23 17:36:43,253] Trial 4 finished with value: 0.9460000000000001 and parameters: {'epochs': 900, 'batch_size': 128, 'test_ratio': 0.2}. Best is trial 2 with value: 0.9570000000000001.


✅ CTAB-GAN Trial 5 Score: 0.9460 (Similarity: 0.9167, Accuracy: 0.9900)

✅ CTAB-GAN Optimization with Score Fix Complete:
   • Best objective score: 0.9570
   • Best hyperparameters:
     - epochs: 1000
     - batch_size: 64
     - test_ratio: 0.1500

📊 CTAB-GAN hyperparameter optimization with score extraction fix completed!
🎯 Expected: Variable scores reflecting actual ML accuracy performance


#### 4.1.3 CTAB-GAN+ Hyperparameter Optimization

In [22]:
# Code Chunk ID: CHUNK_4_1_3_A
# Import required libraries for CTAB-GAN+ optimization
import optuna
import numpy as np
import pandas as pd
from src.models.model_factory import ModelFactory
from src.evaluation.trts_framework import TRTSEvaluator

# CORRECTED CTAB-GAN+ Search Space (3 supported parameters only)
def ctabganplus_search_space(trial):
    """Realistic CTAB-GAN+ hyperparameter space - ONLY supported parameters"""
    return {
        'epochs': trial.suggest_int('epochs', 150, 1000, step=50),  # Slightly higher range for "plus" version
        'batch_size': trial.suggest_categorical('batch_size', [64, 128, 256, 512]),  # Add 512 for enhanced version
        'test_ratio': trial.suggest_float('test_ratio', 0.10, 0.25, step=0.05),  # Slightly wider range
        # REMOVED: All "enhanced" parameters (not supported by constructor)
    }

def ctabganplus_objective(trial):
    """FINAL CORRECTED CTAB-GAN+ objective function with SCORE EXTRACTION FIX"""
    try:
        # Get realistic hyperparameters from trial
        params = ctabganplus_search_space(trial)
        
        print(f"\n🔄 CTAB-GAN+ Trial {trial.number + 1}: epochs={params['epochs']}, batch_size={params['batch_size']}, test_ratio={params['test_ratio']:.3f}")
        
        # Initialize CTAB-GAN+ using ModelFactory
        model = ModelFactory.create("ctabganplus", random_state=42)
        
        # Only pass supported parameters to train()
        result = model.train(data, 
                           epochs=params['epochs'],
                           batch_size=params['batch_size'],
                           test_ratio=params['test_ratio'])
        
        print(f"🏋️ Training CTAB-GAN+ with corrected parameters...")
        
        # Generate synthetic data for evaluation
        synthetic_data = model.generate(len(data))
        
        # CRITICAL FIX: Convert synthetic data labels to match original data types before TRTS evaluation
        synthetic_data_converted = synthetic_data.copy()
        if target_column in synthetic_data_converted.columns and target_column in data.columns:
            # Convert string labels to numeric to match original data type
            if synthetic_data_converted[target_column].dtype == 'object' and data[target_column].dtype != 'object':
                print(f"🔧 Converting synthetic labels from {synthetic_data_converted[target_column].dtype} to {data[target_column].dtype}")
                synthetic_data_converted[target_column] = pd.to_numeric(synthetic_data_converted[target_column], errors='coerce')
                
                # Handle any conversion failures
                if synthetic_data_converted[target_column].isna().any():
                    print(f"⚠️ Some labels failed conversion - filling with mode")
                    mode_value = data[target_column].mode()[0]
                    synthetic_data_converted[target_column].fillna(mode_value, inplace=True)
                
                # Ensure same data type as original
                synthetic_data_converted[target_column] = synthetic_data_converted[target_column].astype(data[target_column].dtype)
                print(f"✅ Label conversion successful: {synthetic_data_converted[target_column].dtype}")
        
        # Calculate similarity score using TRTS framework with converted data
        trts = TRTSEvaluator(random_state=42)
        trts_results = trts.evaluate_trts_scenarios(data, synthetic_data_converted, target_column=target_column)
        
        # 🎯 CRITICAL FIX: Correct Score Extraction (targeting ML accuracy scores, not percentages)
        if 'trts_scores' in trts_results and isinstance(trts_results['trts_scores'], dict):
            trts_scores = list(trts_results['trts_scores'].values())  # Extract ML accuracy scores (0-1 scale)
            print(f"🎯 CORRECTED: ML accuracy scores = {trts_scores}")
        else:
            # Fallback to filtered method if structure unexpected
            print(f"⚠️ Using fallback score extraction")
            trts_scores = [score for score in trts_results.values() if isinstance(score, (int, float)) and 0 <= score <= 1]
            print(f"🔍 Fallback extracted scores = {trts_scores}")
        
        # CORRECTED EVALUATION FAILURE DETECTION (using proper 0-1 scale)
        if not trts_scores:
            print(f"❌ TRTS evaluation failure: NO NUMERIC SCORES RETURNED")
            return 0.0
        elif all(score >= 0.99 for score in trts_scores):  # Now checking 0-1 scale scores
            print(f"❌ TRTS evaluation failure: ALL SCORES ≥0.99 (suspicious perfect scores)")
            print(f"   • Perfect scores detected: {trts_scores}")
            return 0.0  
        else:
            # TRTS evaluation successful
            similarity_score = np.mean(trts_scores) if trts_scores else 0.0
            similarity_score = max(0.0, min(1.0, similarity_score))
            print(f"✅ TRTS evaluation successful: {similarity_score:.4f} (from {len(trts_scores)} ML accuracy scores)")
        
        # Calculate accuracy with converted labels
        try:
            from sklearn.ensemble import RandomForestClassifier
            from sklearn.metrics import accuracy_score
            from sklearn.model_selection import train_test_split
            
            # Use converted synthetic data for accuracy calculation
            if target_column in data.columns and target_column in synthetic_data_converted.columns:
                X_real = data.drop(target_column, axis=1)
                y_real = data[target_column]
                X_synth = synthetic_data_converted.drop(target_column, axis=1) 
                y_synth = synthetic_data_converted[target_column]
                
                # Train on synthetic, test on real (TRTS approach)
                X_train, X_test, y_train, y_test = train_test_split(X_real, y_real, test_size=0.2, random_state=42)
                
                clf = RandomForestClassifier(random_state=42, n_estimators=50)
                clf.fit(X_synth, y_synth)
                
                predictions = clf.predict(X_test)
                accuracy = accuracy_score(y_test, predictions)
                
                # Combined score (weighted average of similarity and accuracy)
                score = 0.6 * similarity_score + 0.4 * accuracy
                score = max(0.0, min(1.0, score))  # Ensure 0-1 range
                
                print(f"✅ CTAB-GAN+ Trial {trial.number + 1} Score: {score:.4f} (Similarity: {similarity_score:.4f}, Accuracy: {accuracy:.4f})")
            else:
                score = similarity_score
                print(f"✅ CTAB-GAN+ Trial {trial.number + 1} Score: {score:.4f} (Similarity: {similarity_score:.4f})")
                
        except Exception as e:
            print(f"⚠️ Accuracy calculation failed: {e}")
            score = similarity_score
            print(f"✅ CTAB-GAN+ Trial {trial.number + 1} Score: {score:.4f} (Similarity: {similarity_score:.4f})")
        
        return score
        
    except Exception as e:
        print(f"❌ CTAB-GAN+ trial {trial.number + 1} failed: {str(e)}")
        return 0.0  # FAILED MODELS RETURN 0.0, NOT 1.0

# Execute CTAB-GAN+ hyperparameter optimization with SCORE EXTRACTION FIX
print("\n🎯 Starting CTAB-GAN+ Hyperparameter Optimization - SCORE EXTRACTION FIX")
print("   • Search space: 3 supported parameters (epochs, batch_size, test_ratio)")
print("   • Enhanced ranges: Slightly higher epochs and wider test_ratio range")
print("   • Parameter validation: Only constructor-supported parameters")
print("   • 🎯 CRITICAL FIX: Correct ML accuracy score extraction (0-1 scale)")
print("   • Proper threshold detection: Using 0-1 scale for perfect score detection")
print("   • Number of trials: 5")
print(f"   • Algorithm: TPE with median pruning")

# Create and execute study
ctabganplus_study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
ctabganplus_study.optimize(ctabganplus_objective, n_trials=5)

# Display results
print(f"\n✅ CTAB-GAN+ Optimization with Score Fix Complete:")
print(f"   • Best objective score: {ctabganplus_study.best_value:.4f}")
print(f"   • Best hyperparameters:")
for key, value in ctabganplus_study.best_params.items():
    if isinstance(value, float):
        print(f"     - {key}: {value:.4f}")
    else:
        print(f"     - {key}: {value}")

# Store best parameters for later use
ctabganplus_best_params = ctabganplus_study.best_params
print("\n📊 CTAB-GAN+ hyperparameter optimization with score extraction fix completed!")
print(f"🎯 Expected: Variable scores reflecting actual ML accuracy performance")

[I 2025-09-23 17:36:43,285] A new study created in memory with name: no-name-143efa27-16de-4cfa-ae47-ae96f0583bef



🎯 Starting CTAB-GAN+ Hyperparameter Optimization - SCORE EXTRACTION FIX
   • Search space: 3 supported parameters (epochs, batch_size, test_ratio)
   • Enhanced ranges: Slightly higher epochs and wider test_ratio range
   • Parameter validation: Only constructor-supported parameters
   • 🎯 CRITICAL FIX: Correct ML accuracy score extraction (0-1 scale)
   • Proper threshold detection: Using 0-1 scale for perfect score detection
   • Number of trials: 5
   • Algorithm: TPE with median pruning

🔄 CTAB-GAN+ Trial 1: epochs=350, batch_size=512, test_ratio=0.200


100%|██████████| 350/350 [03:38<00:00,  1.60it/s]


Finished training in 219.47028398513794  seconds.
🏋️ Training CTAB-GAN+ with corrected parameters...
🎯 CORRECTED: ML accuracy scores = [0.9733333333333334, 0.94, 0.92, 0.96]
✅ TRTS evaluation successful: 0.9483 (from 4 ML accuracy scores)


[I 2025-09-23 17:40:23,014] Trial 0 finished with value: 0.961 and parameters: {'epochs': 350, 'batch_size': 512, 'test_ratio': 0.2}. Best is trial 0 with value: 0.961.


...

✅ CTAB-GAN+ Trial 3 Score: 0.9530 (Similarity: 0.9283, Accuracy: 0.9900)

🔄 CTAB-GAN+ Trial 4: epochs=550, batch_size=64, test_ratio=0.250


100%|██████████| 550/550 [05:12<00:00,  1.76it/s]
[I 2025-09-23 17:57:14,109] Trial 3 finished with value: 0.977 and parameters: {'epochs': 550, 'batch_size': 64, 'test_ratio': 0.25}. Best is trial 3 with value: 0.977.


Finished training in 313.9458932876587  seconds.
🏋️ Training CTAB-GAN+ with corrected parameters...
🎯 CORRECTED: ML accuracy scores = [0.9733333333333334, 0.98, 0.96, 0.9866666666666667]
✅ TRTS evaluation successful: 0.9750 (from 4 ML accuracy scores)
✅ CTAB-GAN+ Trial 4 Score: 0.9770 (Similarity: 0.9750, Accuracy: 0.9800)

🔄 CTAB-GAN+ Trial 5: epochs=700, batch_size=128, test_ratio=0.250


100%|██████████| 700/700 [03:47<00:00,  3.08it/s]
[I 2025-09-23 18:01:02,355] Trial 4 finished with value: 0.9590000000000001 and parameters: {'epochs': 700, 'batch_size': 128, 'test_ratio': 0.25}. Best is trial 3 with value: 0.977.


Finished training in 228.05782985687256  seconds.
🏋️ Training CTAB-GAN+ with corrected parameters...
🎯 CORRECTED: ML accuracy scores = [0.9733333333333334, 0.9133333333333333, 0.9333333333333333, 0.96]
✅ TRTS evaluation successful: 0.9450 (from 4 ML accuracy scores)
✅ CTAB-GAN+ Trial 5 Score: 0.9590 (Similarity: 0.9450, Accuracy: 0.9800)

✅ CTAB-GAN+ Optimization with Score Fix Complete:
   • Best objective score: 0.9770
   • Best hyperparameters:
     - epochs: 550
     - batch_size: 64
     - test_ratio: 0.2500

📊 CTAB-GAN+ hyperparameter optimization with score extraction fix completed!
🎯 Expected: Variable scores reflecting actual ML accuracy performance


#### 4.1.4 GANerAid Hyperparameter Optimization

In [23]:
# Code Chunk ID: CHUNK_4_1_4_A
# GANerAid Search Space and Hyperparameter Optimization

def ganeraid_search_space(trial):
    """
    GENERALIZED GANerAid hyperparameter search space with dynamic constraint adjustment.
    
    CRITICAL INSIGHT: Following CTGAN's compatible_pac pattern for robust constraint handling.
    GANerAid requires: batch_size % nr_of_rows == 0 AND nr_of_rows < dataset_size
    """
    
    # Define available batch sizes (easily extensible like CTGAN)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 100, 128, 200, 256, 400, 500])
    
    # Suggest nr_of_rows from a reasonable range (will be adjusted at runtime)
    max_nr_of_rows = min(50, batch_size // 2)  # Conservative upper bound
    nr_of_rows = trial.suggest_int('nr_of_rows', 4, max_nr_of_rows)
    
    return {
        'epochs': trial.suggest_int('epochs', 100, 500, step=50),  # REDUCED for troubleshooting
        'batch_size': batch_size,
        'nr_of_rows': nr_of_rows,  # Will be adjusted in objective function
        'lr_d': trial.suggest_loguniform('lr_d', 1e-6, 5e-3),
        'lr_g': trial.suggest_loguniform('lr_g', 1e-6, 5e-3),
        'hidden_feature_space': trial.suggest_categorical('hidden_feature_space', [
            100, 150, 200, 300, 400, 500, 600
        ]),
        'binary_noise': trial.suggest_uniform('binary_noise', 0.05, 0.6),
        'generator_decay': trial.suggest_loguniform('generator_decay', 1e-8, 1e-3),
        'discriminator_decay': trial.suggest_loguniform('discriminator_decay', 1e-8, 1e-3),
        'dropout_generator': trial.suggest_uniform('dropout_generator', 0.0, 0.5),
        'dropout_discriminator': trial.suggest_uniform('dropout_discriminator', 0.0, 0.5)
    }

def find_compatible_nr_of_rows(batch_size, requested_nr_of_rows, dataset_size, hidden_feature_space):
    """
    Find largest compatible nr_of_rows <= requested_nr_of_rows that satisfies ALL GANerAid constraints.
    
    DISCOVERED CONSTRAINTS:
    1. batch_size % nr_of_rows == 0 (divisibility for batching)
    2. nr_of_rows < dataset_size (avoid dataset index out of bounds)
    3. hidden_feature_space % nr_of_rows == 0 (CRITICAL: for internal LSTM step calculation)
    4. nr_of_rows >= 4 (reasonable minimum for GANerAid)
    
    From GANerAid model.py line 90: step = int(hidden_size / rows)
    The loop runs 'rows' times, accessing output[:, c, :] where c goes from 0 to rows-1
    """
    
    # Start with requested value and work downward
    compatible_nr_of_rows = requested_nr_of_rows
    
    while compatible_nr_of_rows >= 4:
        # Check all constraints
        batch_divisible = batch_size % compatible_nr_of_rows == 0
        size_safe = compatible_nr_of_rows < dataset_size
        hidden_divisible = hidden_feature_space % compatible_nr_of_rows == 0  # CRITICAL NEW CONSTRAINT
        
        if batch_divisible and size_safe and hidden_divisible:
            return compatible_nr_of_rows
            
        compatible_nr_of_rows -= 1
    
    # If no compatible value found, return 4 as fallback (most likely to work)
    return 4

def ganeraid_objective(trial):
    """GENERALIZED GANerAid objective function with ALL constraint validation."""
    try:
        # Get hyperparameters from trial
        params = ganeraid_search_space(trial)
        
        # DYNAMIC CONSTRAINT ADJUSTMENT (following CTGAN pattern)
        batch_size = params['batch_size']
        requested_nr_of_rows = params['nr_of_rows']
        hidden_feature_space = params['hidden_feature_space']
        dataset_size = len(data) if 'data' in globals() else 569  # fallback
        
        # Find compatible nr_of_rows using runtime adjustment with ALL constraints
        compatible_nr_of_rows = find_compatible_nr_of_rows(batch_size, requested_nr_of_rows, dataset_size, hidden_feature_space)
        
        # Update parameters with compatible value
        if compatible_nr_of_rows != requested_nr_of_rows:
            print(f"🔧 nr_of_rows adjusted: {requested_nr_of_rows} → {compatible_nr_of_rows}")
            print(f"   Reason: batch_size={batch_size}, dataset_size={dataset_size}, hidden_feature_space={hidden_feature_space}")
            params['nr_of_rows'] = compatible_nr_of_rows
        
        print(f"\n🔄 GANerAid Trial {trial.number + 1}: epochs={params['epochs']}, batch_size={params['batch_size']}, nr_of_rows={params['nr_of_rows']}, hidden={params['hidden_feature_space']}")
        print(f"✅ COMPLETE Constraint validation:")
        print(f"   • Batch divisibility: {params['batch_size']} % {params['nr_of_rows']} = {params['batch_size'] % params['nr_of_rows']} (should be 0)")
        print(f"   • Size safety: {params['nr_of_rows']} < {dataset_size} = {params['nr_of_rows'] < dataset_size}")
        print(f"   • Hidden divisibility: {params['hidden_feature_space']} % {params['nr_of_rows']} = {params['hidden_feature_space'] % params['nr_of_rows']} (should be 0)")
        print(f"   • LSTM step size: int({params['hidden_feature_space']} / {params['nr_of_rows']}) = {int(params['hidden_feature_space'] / params['nr_of_rows'])}")
        
        # CORRECTED: Ensure TARGET_COLUMN is available with proper global access
        global TARGET_COLUMN
        if TARGET_COLUMN is None:
            # Try to find target column from various sources
            if 'target_column' in globals():
                TARGET_COLUMN = globals()['target_column']
            elif hasattr(data, 'columns') and len(data.columns) > 0:
                TARGET_COLUMN = data.columns[-1]  # Use last column as fallback
                print(f"🔧 Using fallback target column: {TARGET_COLUMN}")
            else:
                print("❌ No target column available - cannot proceed")
                return 0.0
        
        # CRITICAL DEBUG: Check if enhanced_objective_function_v2 is available
        if 'enhanced_objective_function_v2' not in globals():
            print("❌ enhanced_objective_function_v2 not available - cannot evaluate model")
            return 0.0
        
        # Initialize GANerAid using ModelFactory with corrected parameters
        model = ModelFactory.create("ganeraid", random_state=42)
        model.set_config(params)
        
        # ENHANCED ERROR HANDLING: Wrap training in try-catch for constraint violations
        print("🏋️ Training GANerAid with ALL CONSTRAINTS SATISFIED...")
        start_time = time.time()
        
        try:
            model.train(data, epochs=params['epochs'])
            training_time = time.time() - start_time
            print(f"⏱️ Training completed successfully in {training_time:.1f} seconds")
        except IndexError as e:
            if "index" in str(e) and "out of bounds" in str(e):
                print(f"❌ INDEX ERROR STILL OCCURRED: {e}")
                print(f"   batch_size: {params['batch_size']}, nr_of_rows: {params['nr_of_rows']}, dataset_size: {dataset_size}")
                print(f"   hidden_feature_space: {params['hidden_feature_space']}")
                print(f"   batch_divisible: {params['batch_size']} % {params['nr_of_rows']} = {params['batch_size'] % params['nr_of_rows']}")
                print(f"   size_safe: {params['nr_of_rows']} < {dataset_size} = {params['nr_of_rows'] < dataset_size}")
                print(f"   hidden_divisible: {params['hidden_feature_space']} % {params['nr_of_rows']} = {params['hidden_feature_space'] % params['nr_of_rows']}")
                print(f"   lstm_step: {int(params['hidden_feature_space'] / params['nr_of_rows'])}")
                print("   If error persists, GANerAid may have additional undocumented constraints")
                
                # Try to understand the exact error location
                import traceback
                traceback.print_exc()
                
                return 0.0
            else:
                raise  # Re-raise if it's a different IndexError
        except Exception as e:
            print(f"❌ Training failed with error: {e}")
            return 0.0
        
        # Generate synthetic data
        try:
            synthetic_data = model.generate(len(data))
            print(f"📊 Generated synthetic data: {synthetic_data.shape}")
        except Exception as e:
            print(f"❌ Generation failed with error: {e}")
            return 0.0
        
        # ENHANCED EVALUATION with NaN handling and FIXED pandas.isfinite issue
        try:
            score, similarity_score, accuracy_score = enhanced_objective_function_v2(
                data, synthetic_data, TARGET_COLUMN
            )
            
            print(f"📊 Raw evaluation scores - Similarity: {similarity_score}, Accuracy: {accuracy_score}, Combined: {score}")
            
            # CRITICAL FIX: Handle NaN values which cause trial failures (use np.isfinite, not pd.isfinite)
            if pd.isna(score) or pd.isna(similarity_score) or pd.isna(accuracy_score):
                print(f"⚠️ NaN detected in scores - similarity: {similarity_score}, accuracy: {accuracy_score}, combined: {score}")
                print("   Replacing NaN values with 0.0 to prevent trial failure")
                
                # Replace NaN with reasonable defaults
                if pd.isna(similarity_score):
                    similarity_score = 0.0
                if pd.isna(accuracy_score):
                    accuracy_score = 0.0
                if pd.isna(score):
                    # Recalculate score if main score is NaN
                    score = 0.6 * similarity_score + 0.4 * accuracy_score
            
            # Ensure score is not NaN or infinite (FIXED: use np.isfinite instead of pd.isfinite)
            if pd.isna(score) or not np.isfinite(score):
                print(f"❌ Final score is invalid: {score}, setting to 0.0")
                score = 0.0
            
            # Clamp score to valid range
            score = max(0.0, min(1.0, score))
            
            print(f"✅ GANerAid Trial {trial.number + 1} FINAL Score: {score:.4f} (Similarity: {similarity_score:.4f}, Accuracy: {accuracy_score:.4f})")
            return float(score)  # Ensure it's a regular float, not numpy float
            
        except Exception as e:
            print(f"❌ Evaluation failed: {e}")
            import traceback
            traceback.print_exc()
            return 0.0
        
    except Exception as e:
        print(f"❌ GANerAid trial {trial.number + 1} failed: {str(e)}")
        import traceback
        traceback.print_exc()
        return 0.0

# Execute GANerAid hyperparameter optimization with COMPLETE constraint handling
print("\n🎯 Starting GANerAid Hyperparameter Optimization - COMPLETE CONSTRAINT DISCOVERY")
print("   • DISCOVERED CRITICAL CONSTRAINT: hidden_feature_space % nr_of_rows == 0")
print("   • FOLLOWING CTGAN PATTERN: Dynamic runtime constraint adjustment")
print("   • EASILY EXTENSIBLE: Add new batch_size values without hardcoding combinations")
print("   • ALL CONSTRAINTS: batch_size % nr_of_rows == 0 AND nr_of_rows < dataset_size AND hidden_feature_space % nr_of_rows == 0")
print("   • EPOCHS: Reduced to 100-500 for troubleshooting")
print("   • ALGORITHM: TPE with median pruning")

# Show the complete constraint handling
print(f"🔧 Complete constraint handling discovered from GANerAid model.py:")
print(f"   • Batch constraint: batch_size % nr_of_rows == 0 (for proper batching)")
print(f"   • Size constraint: nr_of_rows < dataset_size (avoid dataset index errors)")
print(f"   • CRITICAL Hidden constraint: hidden_feature_space % nr_of_rows == 0 (for LSTM step calculation)")
print(f"   • LSTM step formula: int(hidden_feature_space / nr_of_rows)")
print(f"   • Output tensor access: output[:, c, :] where c ranges from 0 to nr_of_rows-1")

# Validate dataset availability before starting optimization
if 'data' not in globals() or data is None:
    print("❌ Dataset not available - cannot perform optimization")
else:
    dataset_info = f"Dataset size: {len(data)}, Columns: {len(data.columns)}"
    print(f"📊 {dataset_info}")
    
    # Demonstrate complete constraint adjustment for current dataset
    print(f"\n🔍 Example COMPLETE constraint adjustments for dataset size {len(data)}:")
    example_cases = [
        (128, 32, 200),  # The failing case from the error
        (64, 16, 200),
        (100, 25, 300),
        (256, 16, 400)
    ]
    
    for batch_size, requested_nr_of_rows, hidden_feature_space in example_cases:
        compatible = find_compatible_nr_of_rows(batch_size, requested_nr_of_rows, len(data), hidden_feature_space)
        adjustment = "" if compatible == requested_nr_of_rows else f" → {compatible}"
        print(f"   • batch_size={batch_size}, hidden={hidden_feature_space}, requested_nr_of_rows={requested_nr_of_rows}{adjustment}")
        print(f"     - Batch check: {batch_size} % {compatible} = {batch_size % compatible}")
        print(f"     - Size check: {compatible} < {len(data)} = {compatible < len(data)}")  
        print(f"     - Hidden check: {hidden_feature_space} % {compatible} = {hidden_feature_space % compatible}")
        print(f"     - LSTM step: {int(hidden_feature_space / compatible)}")
    
    # Create and execute study with enhanced error handling
    try:
        print(f"\n🚀 Starting optimization with {5} trials (increased for troubleshooting as requested)...")
        
        ganeraid_study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
        ganeraid_study.optimize(ganeraid_objective, n_trials=5)  # INCREASED to 5 as requested
        
        # Display results
        print(f"\n✅ GANerAid Optimization Complete:")
        print(f"   • Best objective score: {ganeraid_study.best_value:.4f}")
        print(f"   • Best parameters:")
        for key, value in ganeraid_study.best_params.items():
            if isinstance(value, float):
                print(f"     - {key}: {value:.6f}")
            else:
                print(f"     - {key}: {value}")
        print(f"   • Total trials completed: {len(ganeraid_study.trials)}")
        print(f"   • Successful trials: {len([t for t in ganeraid_study.trials if t.value is not None and t.value > 0])}")
        print(f"   • Failed trials: {len([t for t in ganeraid_study.trials if t.state == optuna.trial.TrialState.FAIL])}")
        
        # Validate the best parameters follow all constraints
        best_params = ganeraid_study.best_params
        if 'batch_size' in best_params and 'nr_of_rows' in best_params and 'hidden_feature_space' in best_params:
            best_batch_size = best_params['batch_size']
            best_nr_of_rows = best_params['nr_of_rows'] 
            best_hidden = best_params['hidden_feature_space']
            
            # Reconstruct the compatible adjustment that would have been used
            dataset_size = len(data)
            compatible_check = find_compatible_nr_of_rows(best_batch_size, best_nr_of_rows, dataset_size, best_hidden)
            
            print(f"   • Best combination COMPLETE validation:")
            print(f"     - batch_size={best_batch_size}, nr_of_rows={best_nr_of_rows}, hidden={best_hidden}")
            print(f"     - Batch divisibility: {best_batch_size} % {best_nr_of_rows} = {best_batch_size % best_nr_of_rows}")
            print(f"     - Size safety: {best_nr_of_rows} < {dataset_size} = {best_nr_of_rows < dataset_size}")
            print(f"     - Hidden divisibility: {best_hidden} % {best_nr_of_rows} = {best_hidden % best_nr_of_rows}")
            print(f"     - Compatible check result: {compatible_check}")
            print(f"     - LSTM step size: {int(best_hidden / best_nr_of_rows)}")
            
            all_constraints_satisfied = (
                best_batch_size % best_nr_of_rows == 0 and
                best_nr_of_rows < dataset_size and
                best_hidden % best_nr_of_rows == 0
            )
            
            if all_constraints_satisfied:
                print("     ✅ Best parameters satisfy ALL discovered constraints")
            else:
                print("     ❌ WARNING: Best parameters show constraint violations")
        
        # Store best parameters for later use
        ganeraid_best_params = ganeraid_study.best_params
        print("\n📊 GANerAid hyperparameter optimization with COMPLETE constraints discovered!")
        print("🎯 BREAKTHROUGH: Found the missing hidden_feature_space % nr_of_rows == 0 constraint")
        print("🎯 Approach: Dynamic runtime adjustment (like CTGAN's compatible_pac)")
        print("🎯 Extensible: Add new batch_size/hidden values without hardcoding combinations")
        print("🎯 DEBUG: Enhanced error reporting and evaluation function validation")
        
    except Exception as e:
        print(f"❌ GANerAid optimization failed: {e}")
        import traceback
        traceback.print_exc()

[I 2025-09-23 18:01:02,388] A new study created in memory with name: no-name-3791557c-06f5-4257-9604-7bfa5fef3695



🎯 Starting GANerAid Hyperparameter Optimization - COMPLETE CONSTRAINT DISCOVERY
   • DISCOVERED CRITICAL CONSTRAINT: hidden_feature_space % nr_of_rows == 0
   • FOLLOWING CTGAN PATTERN: Dynamic runtime constraint adjustment
   • EASILY EXTENSIBLE: Add new batch_size values without hardcoding combinations
   • ALL CONSTRAINTS: batch_size % nr_of_rows == 0 AND nr_of_rows < dataset_size AND hidden_feature_space % nr_of_rows == 0
   • EPOCHS: Reduced to 100-500 for troubleshooting
   • ALGORITHM: TPE with median pruning
🔧 Complete constraint handling discovered from GANerAid model.py:
   • Batch constraint: batch_size % nr_of_rows == 0 (for proper batching)
   • Size constraint: nr_of_rows < dataset_size (avoid dataset index errors)
   • CRITICAL Hidden constraint: hidden_feature_space % nr_of_rows == 0 (for LSTM step calculation)
   • LSTM step formula: int(hidden_feature_space / nr_of_rows)
   • Output tensor access: output[:, c, :] where c ranges from 0 to nr_of_rows-1
📊 Dataset size: 

100%|██████████| 100/100 [00:16<00:00,  5.90it/s, loss=d error: 1.4005330801010132 --- g error 0.6307504773139954]


⏱️ Training completed successfully in 17.0 seconds
Generating 500 samples


[I 2025-09-23 18:01:19,713] Trial 0 finished with value: 0.38876982041264063 and parameters: {'batch_size': 200, 'nr_of_rows': 9, 'epochs': 100, 'lr_d': 4.8097408816366726e-06, 'lr_g': 0.0005869717663047047, 'hidden_feature_space': 400, 'binary_noise': 0.2975316760859171, 'generator_decay': 9.720769472454529e-07, 'discriminator_decay': 4.3056073413243336e-07, 'dropout_generator': 0.39024462744787497, 'dropout_discriminator': 0.022941608148305204}. Best is trial 0 with value: 0.38876982041264063.


...

[OK] TRTS (Synthetic->Real): 0.4580
[OK] TRTS Evaluation: 2 scenarios, Average: 0.2710
[CHART] Combined Score: 0.3536 (Similarity: 0.4086, Accuracy: 0.2710)
📊 Raw evaluation scores - Similarity: 0.4085906288111996, Accuracy: 0.271, Combined: 0.3535543772867198
✅ GANerAid Trial 4 FINAL Score: 0.3536 (Similarity: 0.4086, Accuracy: 0.2710)
🔧 nr_of_rows adjusted: 18 → 4
   Reason: batch_size=256, dataset_size=500, hidden_feature_space=500

🔄 GANerAid Trial 5: epochs=250, batch_size=256, nr_of_rows=4, hidden=500
✅ COMPLETE Constraint validation:
   • Batch divisibility: 256 % 4 = 0 (should be 0)
   • Size safety: 4 < 500 = True
   • Hidden divisibility: 500 % 4 = 0 (should be 0)
   • LSTM step size: int(500 / 4) = 125
🏋️ Training GANerAid with ALL CONSTRAINTS SATISFIED...
Initialized gan with the following parameters: 
lr_d = 0.001857992893806313
lr_g = 0.00013956404896338346
hidden_feature_space = 500
batch_size = 256
nr_of_rows = 4
binary_noise = 0.05206226298620407
Start training of gan 

100%|██████████| 250/250 [01:46<00:00,  2.34it/s, loss=d error: 0.43049274384975433 --- g error 3.943638563156128]  


⏱️ Training completed successfully in 106.7 seconds
Generating 500 samples


[I 2025-09-23 18:06:09,125] Trial 4 finished with value: 0.5575873197717277 and parameters: {'batch_size': 256, 'nr_of_rows': 18, 'epochs': 250, 'lr_d': 0.001857992893806313, 'lr_g': 0.00013956404896338346, 'hidden_feature_space': 500, 'binary_noise': 0.05206226298620407, 'generator_decay': 2.3729141043627496e-06, 'discriminator_decay': 3.553809894782079e-05, 'dropout_generator': 0.4632615902268474, 'dropout_discriminator': 0.4294838645221755}. Best is trial 4 with value: 0.5575873197717277.


📊 Generated synthetic data: (500, 19)
[TARGET] Enhanced objective function using target column: 'Outcome'
[OK] Similarity Analysis: 19/19 valid metrics, Average: 0.4573
[OK] TRTS (Synthetic->Real): 0.8240
[OK] TRTS Evaluation: 2 scenarios, Average: 0.7080
[CHART] Combined Score: 0.5576 (Similarity: 0.4573, Accuracy: 0.7080)
📊 Raw evaluation scores - Similarity: 0.45731219961954617, Accuracy: 0.708, Combined: 0.5575873197717277
✅ GANerAid Trial 5 FINAL Score: 0.5576 (Similarity: 0.4573, Accuracy: 0.7080)

✅ GANerAid Optimization Complete:
   • Best objective score: 0.5576
   • Best parameters:
     - batch_size: 256
     - nr_of_rows: 18
     - epochs: 250
     - lr_d: 0.001858
     - lr_g: 0.000140
     - hidden_feature_space: 500
     - binary_noise: 0.052062
     - generator_decay: 0.000002
     - discriminator_decay: 0.000036
     - dropout_generator: 0.463262
     - dropout_discriminator: 0.429484
   • Total trials completed: 5
   • Successful trials: 5
   • Failed trials: 0
   • B

#### 4.1.5 CopulaGAN Hyperparameter Optimization

In [24]:
# Code Chunk ID: CHUNK_4_1_5_A
# CopulaGAN Search Space and Hyperparameter Optimization

# Import required libraries at the top
import optuna
import time
from src.models.model_factory import ModelFactory
from setup import enhanced_objective_function_v2

def copulagan_search_space(trial):
    """
    GENERALIZED CopulaGAN hyperparameter search space with dynamic constraint adjustment.
    
    CRITICAL INSIGHT: Following CTGAN's compatible_pac pattern for robust constraint handling.
    CopulaGAN requires discrete_columns to be properly defined.
    """
    return {
        'epochs': trial.suggest_int('epochs', 50, 500, step=50),
        'batch_size': trial.suggest_categorical('batch_size', [100, 200, 500]),
        'generator_lr': trial.suggest_loguniform('generator_lr', 1e-5, 1e-2),
        'discriminator_lr': trial.suggest_loguniform('discriminator_lr', 1e-5, 1e-2),
        'generator_decay': trial.suggest_loguniform('generator_decay', 1e-8, 1e-4),
        'discriminator_decay': trial.suggest_loguniform('discriminator_decay', 1e-8, 1e-4),
    }

def copulagan_objective(trial):
    """GENERALIZED CopulaGAN objective function."""
    try:
        # Get hyperparameters from trial
        params = copulagan_search_space(trial)
        
        print(f"\n🔄 CopulaGAN Trial {trial.number + 1}: epochs={params['epochs']}, batch_size={params['batch_size']}")
        
        # Initialize CopulaGAN using ModelFactory
        model = ModelFactory.create("copulagan", random_state=42)
        
        # Auto-detect discrete columns for CopulaGAN
        discrete_columns = data.select_dtypes(include=['object']).columns.tolist()
        print(f"📊 Detected discrete columns: {discrete_columns}")
        
        # Train model
        print(f"🏋️ Training CopulaGAN...")
        start_time = time.time()
        
        try:
            model.train(data, discrete_columns=discrete_columns, **params)
            training_time = time.time() - start_time
            print(f"⏱️ Training completed successfully in {training_time:.1f} seconds")
        except Exception as e:
            print(f"❌ Training failed: {str(e)}")
            return 0.0

        # Generate synthetic data for evaluation
        synthetic_data = model.generate(5000)
        print(f"📊 Generated synthetic data: {synthetic_data.shape}")
        
        # Use enhanced objective function
        combined_score, similarity_score, accuracy_score = enhanced_objective_function_v2(
            data, synthetic_data, TARGET_COLUMN
        )
        
        print(f"🎯 Trial {trial.number + 1} Results:")
        print(f"   • Combined Score: {combined_score:.4f}")
        print(f"   • Similarity: {similarity_score:.4f}")
        print(f"   • Accuracy: {accuracy_score:.4f}")
        
        return combined_score
        
    except Exception as e:
        print(f"❌ Trial {trial.number + 1} failed: {str(e)}")
        import traceback
        traceback.print_exc()
        return 0.0

# Create and run optimization study
print(f"\n🔧 SECTION 4.5: CopulaGAN HYPERPARAMETER OPTIMIZATION")
print("=" * 80)
print(f"🔄 Creating CopulaGAN optimization study...")
print(f"📊 Dataset info: {len(data)} rows, {len(data.columns)} columns")
print(f"📊 Target column '{TARGET_COLUMN}' unique values: {data[TARGET_COLUMN].nunique()}")
print()

# Create study and optimize
copulagan_study = optuna.create_study(direction='maximize')
copulagan_study.optimize(copulagan_objective, n_trials=5)

# Extract and display results
print(f"\n🎯 ============================================================================")
print(f"🎯 CopulaGAN OPTIMIZATION COMPLETE!")
print(f"🎯 ============================================================================")

best_trial = copulagan_study.best_trial
best_params_copulagan = best_trial.params
best_score_copulagan = best_trial.value

print(f"🏆 Best CopulaGAN Trial: {best_trial.number + 1}")
print(f"📈 Best Score: {best_score_copulagan:.4f}")
print(f"⚙️ Best Parameters:")
for param, value in best_params_copulagan.items():
    print(f"   • {param}: {value}")

# Store results in standardized format for Section 5.5
copulagan_results = {
    'model_name': 'CopulaGAN',
    'best_score': best_score_copulagan,
    'best_params': best_params_copulagan,
    'study': copulagan_study,
    'n_trials': len(copulagan_study.trials)
}

print(f"💾 Results stored for Section 5.5 final model training")
print("=" * 80)

[I 2025-09-23 18:06:09,133] A new study created in memory with name: no-name-286ce7e1-fae9-4470-918a-e7d2db568135



🔧 SECTION 4.5: CopulaGAN HYPERPARAMETER OPTIMIZATION
🔄 Creating CopulaGAN optimization study...
📊 Dataset info: 500 rows, 19 columns
📊 Target column 'Outcome' unique values: 2


🔄 CopulaGAN Trial 1: epochs=450, batch_size=200
📊 Detected discrete columns: []
🏋️ Training CopulaGAN...
⏱️ Training completed successfully in 29.9 seconds
📊 Generated synthetic data: (5000, 19)
[TARGET] Enhanced objective function using target column: 'Outcome'
[OK] Similarity Analysis: 19/19 valid metrics, Average: 0.5245


[I 2025-09-23 18:06:39,875] Trial 0 finished with value: 0.6346353992657503 and parameters: {'epochs': 450, 'batch_size': 200, 'generator_lr': 0.005509443774394306, 'discriminator_lr': 2.228666725282162e-05, 'generator_decay': 1.6780689868300118e-08, 'discriminator_decay': 2.767655728977855e-07}. Best is trial 0 with value: 0.6346353992657503.


[OK] TRTS (Synthetic->Real): 0.9420
[OK] TRTS Evaluation: 2 scenarios, Average: 0.7999
[CHART] Combined Score: 0.6346 (Similarity: 0.5245, Accuracy: 0.7999)
🎯 Trial 1 Results:
   • Combined Score: 0.6346
   • Similarity: 0.5245
   • Accuracy: 0.7999

🔄 CopulaGAN Trial 2: epochs=250, batch_size=200
📊 Detected discrete columns: []
🏋️ Training CopulaGAN...
⏱️ Training completed successfully in 13.9 seconds
📊 Generated synthetic data: (5000, 19)
[TARGET] Enhanced objective function using target column: 'Outcome'
[OK] Similarity Analysis: 19/19 valid metrics, Average: 0.5285


[I 2025-09-23 18:06:54,681] Trial 1 finished with value: 0.5082088484709523 and parameters: {'epochs': 250, 'batch_size': 200, 'generator_lr': 3.497623493327935e-05, 'discriminator_lr': 0.007218666116899715, 'generator_decay': 3.026708345508104e-07, 'discriminator_decay': 8.519494339006559e-07}. Best is trial 0 with value: 0.6346353992657503.


...

[OK] TRTS (Synthetic->Real): 0.5320
[OK] TRTS Evaluation: 2 scenarios, Average: 0.5622
[CHART] Combined Score: 0.5288 (Similarity: 0.5065, Accuracy: 0.5622)
🎯 Trial 3 Results:
   • Combined Score: 0.5288
   • Similarity: 0.5065
   • Accuracy: 0.5622

🔄 CopulaGAN Trial 4: epochs=200, batch_size=200
📊 Detected discrete columns: []
🏋️ Training CopulaGAN...
⏱️ Training completed successfully in 11.1 seconds
📊 Generated synthetic data: (5000, 19)
[TARGET] Enhanced objective function using target column: 'Outcome'
[OK] Similarity Analysis: 19/19 valid metrics, Average: 0.5728


[I 2025-09-23 18:07:24,011] Trial 3 finished with value: 0.5653805722071277 and parameters: {'epochs': 200, 'batch_size': 200, 'generator_lr': 0.0015968631024842889, 'discriminator_lr': 7.070836260617859e-05, 'generator_decay': 1.1915740032885193e-06, 'discriminator_decay': 2.1176530762830528e-08}. Best is trial 0 with value: 0.6346353992657503.


[OK] TRTS (Synthetic->Real): 0.5960
[OK] TRTS Evaluation: 2 scenarios, Average: 0.5543
[CHART] Combined Score: 0.5654 (Similarity: 0.5728, Accuracy: 0.5543)
🎯 Trial 4 Results:
   • Combined Score: 0.5654
   • Similarity: 0.5728
   • Accuracy: 0.5543

🔄 CopulaGAN Trial 5: epochs=500, batch_size=200
📊 Detected discrete columns: []
🏋️ Training CopulaGAN...
⏱️ Training completed successfully in 25.3 seconds
📊 Generated synthetic data: (5000, 19)
[TARGET] Enhanced objective function using target column: 'Outcome'
[OK] Similarity Analysis: 19/19 valid metrics, Average: 0.5379


[I 2025-09-23 18:07:50,107] Trial 4 finished with value: 0.6619579117802629 and parameters: {'epochs': 500, 'batch_size': 200, 'generator_lr': 0.003930639677378101, 'discriminator_lr': 0.0027708626955453837, 'generator_decay': 2.0075580675608644e-05, 'discriminator_decay': 6.400053058655845e-05}. Best is trial 4 with value: 0.6619579117802629.


[OK] TRTS (Synthetic->Real): 0.9380
[OK] TRTS Evaluation: 2 scenarios, Average: 0.8480
[CHART] Combined Score: 0.6620 (Similarity: 0.5379, Accuracy: 0.8480)
🎯 Trial 5 Results:
   • Combined Score: 0.6620
   • Similarity: 0.5379
   • Accuracy: 0.8480

🎯 ============================================================================
🎯 CopulaGAN OPTIMIZATION COMPLETE!
🎯 ============================================================================
🏆 Best CopulaGAN Trial: 5
📈 Best Score: 0.6620
⚙️ Best Parameters:
   • epochs: 500
   • batch_size: 200
   • generator_lr: 0.003930639677378101
   • discriminator_lr: 0.0027708626955453837
   • generator_decay: 2.0075580675608644e-05
   • discriminator_decay: 6.400053058655845e-05
💾 Results stored for Section 5.5 final model training


#### 4.1.6 TVAE Hyperparameter Optimization

In [25]:
# Code Chunk ID: CHUNK_4_1_6_A
# TVAE Robust Search Space (from hypertuning_eg.md)
def tvae_search_space(trial):
    return {
        "epochs": trial.suggest_int("epochs", 50, 500, step=50),  # Training cycles
        "batch_size": trial.suggest_categorical("batch_size", [64, 128, 256, 512]),  # Training batch size
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e-2),  # Learning rate
        "compress_dims": trial.suggest_categorical(  # Encoder architecture
            "compress_dims", [[128, 128], [256, 128], [256, 128, 64]]
        ),
        "decompress_dims": trial.suggest_categorical(  # Decoder architecture
            "decompress_dims", [[128, 128], [64, 128], [64, 128, 256]]
        ),
        "embedding_dim": trial.suggest_int("embedding_dim", 32, 256, step=32),  # Latent space bottleneck size
        "l2scale": trial.suggest_loguniform("l2scale", 1e-6, 1e-2),  # L2 regularization weight
        "dropout": trial.suggest_uniform("dropout", 0.0, 0.5),  # Dropout probability
        "log_frequency": trial.suggest_categorical("log_frequency", [True, False]),  # Use log frequency for representation
        "conditional_generation": trial.suggest_categorical("conditional_generation", [True, False]),  # Conditioned generation
        "verbose": trial.suggest_categorical("verbose", [True])
    }

# TVAE Objective Function using robust search space
def tvae_objective(trial):
    params = tvae_search_space(trial)
    
    try:
        print(f"\n🔄 TVAE Trial {trial.number + 1}: epochs={params['epochs']}, batch_size={params['batch_size']}, lr={params['learning_rate']:.2e}")
        
        # Initialize TVAE using ModelFactory with robust params
        model = ModelFactory.create("TVAE", random_state=42)
        model.set_config(params)
        
        # Train model
        print("🏋️ Training TVAE...")
        start_time = time.time()
        model.train(data, **params)
        training_time = time.time() - start_time
        print(f"⏱️ Training completed in {training_time:.1f} seconds")
        
        # Generate synthetic data
        synthetic_data = model.generate(len(data))
        
        # Evaluate using enhanced objective function
        score, similarity_score, accuracy_score = enhanced_objective_function_v2(data, synthetic_data, target_column)
        
        print(f"✅ TVAE Trial {trial.number + 1} Score: {score:.4f} (Similarity: {similarity_score:.4f}, Accuracy: {accuracy_score:.4f})")
        
        return score
        
    except Exception as e:
        print(f"❌ TVAE trial {trial.number + 1} failed: {str(e)}")
        return 0.0

# Execute TVAE hyperparameter optimization
print("\n🎯 Starting TVAE Hyperparameter Optimization")
print(f"   • Search space: 5 parameters")
print(f"   • Number of trials: 5")
print(f"   • Algorithm: TPE with median pruning")

# Create and execute study
tvae_study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
tvae_study.optimize(tvae_objective, n_trials=5)

# Display results
print(f"\n✅ TVAE Optimization Complete:")
print(f"Best score: {tvae_study.best_value:.4f}")
print(f"Best params: {tvae_study.best_params}")

# Store best parameters
tvae_best_params = tvae_study.best_params
print("\n📊 TVAE hyperparameter optimization completed successfully!")

[I 2025-09-23 18:07:50,114] A new study created in memory with name: no-name-6291d9c0-a3ce-43c7-9254-a94e5e17a352



🎯 Starting TVAE Hyperparameter Optimization
   • Search space: 5 parameters
   • Number of trials: 5
   • Algorithm: TPE with median pruning

🔄 TVAE Trial 1: epochs=100, batch_size=512, lr=2.57e-03
🏋️ Training TVAE...
⏱️ Training completed in 2.7 seconds
[TARGET] Enhanced objective function using target column: 'Outcome'
[OK] Similarity Analysis: 19/19 valid metrics, Average: 0.5138


[I 2025-09-23 18:07:52,997] Trial 0 finished with value: 0.6862953810052159 and parameters: {'epochs': 100, 'batch_size': 512, 'learning_rate': 0.0025737965127029523, 'compress_dims': [128, 128], 'decompress_dims': [64, 128, 256], 'embedding_dim': 224, 'l2scale': 1.3565274063516623e-05, 'dropout': 0.4852365272653375, 'log_frequency': False, 'conditional_generation': False, 'verbose': True}. Best is trial 0 with value: 0.6862953810052159.


[OK] TRTS (Synthetic->Real): 0.9800
[OK] TRTS Evaluation: 2 scenarios, Average: 0.9450
[CHART] Combined Score: 0.6863 (Similarity: 0.5138, Accuracy: 0.9450)
✅ TVAE Trial 1 Score: 0.6863 (Similarity: 0.5138, Accuracy: 0.9450)

🔄 TVAE Trial 2: epochs=250, batch_size=512, lr=3.67e-03
🏋️ Training TVAE...
⏱️ Training completed in 4.8 seconds
[TARGET] Enhanced objective function using target column: 'Outcome'
[OK] Similarity Analysis: 19/19 valid metrics, Average: 0.5368


[I 2025-09-23 18:07:57,997] Trial 1 finished with value: 0.6888637647863305 and parameters: {'epochs': 250, 'batch_size': 512, 'learning_rate': 0.0036744637184711413, 'compress_dims': [256, 128, 64], 'decompress_dims': [128, 128], 'embedding_dim': 96, 'l2scale': 0.003581786967140014, 'dropout': 0.2793669036439709, 'log_frequency': True, 'conditional_generation': False, 'verbose': True}. Best is trial 1 with value: 0.6888637647863305.


...

[OK] TRTS (Synthetic->Real): 0.9780
[OK] TRTS Evaluation: 2 scenarios, Average: 0.9660
[CHART] Combined Score: 0.7587 (Similarity: 0.6204, Accuracy: 0.9660)
✅ TVAE Trial 3 Score: 0.7587 (Similarity: 0.6204, Accuracy: 0.9660)

🔄 TVAE Trial 4: epochs=250, batch_size=256, lr=2.77e-04
🏋️ Training TVAE...


[I 2025-09-23 18:08:22,707] Trial 3 finished with value: 0.7258089149519298 and parameters: {'epochs': 250, 'batch_size': 256, 'learning_rate': 0.00027717229683907183, 'compress_dims': [128, 128], 'decompress_dims': [64, 128, 256], 'embedding_dim': 160, 'l2scale': 1.3257619240952296e-06, 'dropout': 0.49823428495288896, 'log_frequency': False, 'conditional_generation': False, 'verbose': True}. Best is trial 2 with value: 0.7586682819465667.


⏱️ Training completed in 6.5 seconds
[TARGET] Enhanced objective function using target column: 'Outcome'
[OK] Similarity Analysis: 19/19 valid metrics, Average: 0.5670
[OK] TRTS (Synthetic->Real): 0.9720
[OK] TRTS Evaluation: 2 scenarios, Average: 0.9640
[CHART] Combined Score: 0.7258 (Similarity: 0.5670, Accuracy: 0.9640)
✅ TVAE Trial 4 Score: 0.7258 (Similarity: 0.5670, Accuracy: 0.9640)

🔄 TVAE Trial 5: epochs=150, batch_size=256, lr=4.04e-04
🏋️ Training TVAE...
⏱️ Training completed in 4.4 seconds
[TARGET] Enhanced objective function using target column: 'Outcome'
[OK] Similarity Analysis: 19/19 valid metrics, Average: 0.5657


[I 2025-09-23 18:08:27,366] Trial 4 finished with value: 0.7266378851721145 and parameters: {'epochs': 150, 'batch_size': 256, 'learning_rate': 0.00040361115200459767, 'compress_dims': [256, 128], 'decompress_dims': [64, 128, 256], 'embedding_dim': 64, 'l2scale': 1.6713913953787254e-06, 'dropout': 0.4859851935062082, 'log_frequency': True, 'conditional_generation': False, 'verbose': True}. Best is trial 2 with value: 0.7586682819465667.


[OK] TRTS (Synthetic->Real): 0.9820
[OK] TRTS Evaluation: 2 scenarios, Average: 0.9680
[CHART] Combined Score: 0.7266 (Similarity: 0.5657, Accuracy: 0.9680)
✅ TVAE Trial 5 Score: 0.7266 (Similarity: 0.5657, Accuracy: 0.9680)

✅ TVAE Optimization Complete:
Best score: 0.7587
Best params: {'epochs': 450, 'batch_size': 128, 'learning_rate': 0.0033436005261178008, 'compress_dims': [256, 128, 64], 'decompress_dims': [64, 128, 256], 'embedding_dim': 96, 'l2scale': 0.0001437927714752794, 'dropout': 0.0917106087887542, 'log_frequency': True, 'conditional_generation': True, 'verbose': True}

📊 TVAE hyperparameter optimization completed successfully!


### 4.2 Batch process 

This section outputs figures and graphics from models run in 4.1. The next chunk will output results for whatever subsections of 4.1 were run. I.e., if there's need to debug one method, there's no need to run all subsections of 4.1.

In [26]:
# Code Chunk ID: CHUNK_4_2_0_A
# ============================================================================
# SECTION 4 - BATCH HYPERPARAMETER OPTIMIZATION ANALYSIS
# ============================================================================

print("🔍 SECTION 4 - HYPERPARAMETER OPTIMIZATION BATCH ANALYSIS")
print("=" * 80)
print()

# Use enhanced batch evaluation function from setup.py
# Following exact same pattern as CHUNK_018 (Section 3) - no module reload needed!
try:
    # Run batch analysis with file export for all models
    section4_batch_results = evaluate_hyperparameter_optimization_results(
        section_number=4,
        scope=globals(),  # Pass notebook scope to access study variables
        target_column=TARGET_COLUMN
    )
    
    print("\n" + "="*80)
    print("✅ SECTION 4 HYPERPARAMETER OPTIMIZATION BATCH ANALYSIS COMPLETED!")
    print("="*80)
    print(f"📊 Models processed: {len(section4_batch_results['summary_data'])}")
    print(f"📁 Results exported to: {section4_batch_results['results_dir']}")
    print(f"📋 Individual model analysis files:")
    print("   • Hyperparameter parameter_analysis.png plots")
    print("   • Optimization convergence_analysis.png graphs")
    print("   • Parameter correlation matrices")
    print("   • Best trial summary tables")
    print("   • Comprehensive optimization summary CSV")

    
except Exception as e:
    print(f"❌ Batch hyperparameter analysis failed: {str(e)}")
    print(f"🔍 Error details: {type(e).__name__}")
    import traceback
    traceback.print_exc()
    print("\n⚠️  Falling back to individual chunk analysis if needed")

# ============================================================================
# SAVE BEST PARAMETERS TO CSV FOR SECTION 5 USE
# ============================================================================
print("\n" + "=" * 80)
print("💾 SAVING BEST PARAMETERS FROM SECTION 4 OPTIMIZATION")
print("=" * 80)

try:
    # Save all best parameters to CSV using setup.py function
    param_save_results = save_best_parameters_to_csv(
        scope=globals(),
        section_number=4,
        dataset_identifier=DATASET_IDENTIFIER
    )
    
    if param_save_results['success']:
        print(f"\n✅ Parameter saving completed successfully!")
        print(f"   • Files saved: {len(param_save_results['files_saved'])}")
        print(f"   • Parameter entries: {param_save_results['parameters_count']}")
        print(f"   • Models processed: {param_save_results['models_count']}")
        print(f"   • Directory: {param_save_results['results_dir']}")
        
        # Display saved files
        for file_path in param_save_results['files_saved']:
            print(f"     📁 {file_path.split('/')[-1]}")
    else:
        print(f"\n⚠️  Parameter saving completed with issues: {param_save_results['message']}")
        
except Exception as e:
    print(f"\n❌ Parameter saving failed: {str(e)}")
    print(f"   Section 5 will fall back to memory-based parameter retrieval")

print(f"\n📈 Section 4 hyperparameter optimization analysis complete!")
print("🏁 Ready for Section 5: Optimized model re-training")

🔍 SECTION 4 - HYPERPARAMETER OPTIMIZATION BATCH ANALYSIS

[LOCATION] Using DATASET_IDENTIFIER from scope: pakistani-diabetes-dataset
[TARGET] Final DATASET_IDENTIFIER for Section 4: pakistani-diabetes-dataset

SECTION 4 - HYPERPARAMETER OPTIMIZATION BATCH ANALYSIS
[FOLDER] Base results directory: results/pakistani-diabetes-dataset/2025-09-23/Section-4
[TARGET] Target column: Outcome
[CHART] Dataset identifier: pakistani-diabetes-dataset


[SEARCH] 4.1.1: CTGAN Hyperparameter Optimization Analysis
------------------------------------------------------------
[OK] CTGAN optimization study found
[FOLDER] Model directory: results/pakistani-diabetes-dataset/2025-09-23/Section-4/CTGAN
[SEARCH] ANALYZING CTGAN HYPERPARAMETER OPTIMIZATION
[CHART] 1. TRIAL DATA EXTRACTION AND PROCESSING
----------------------------------------
[OK] Extracted 5 trials for analysis
[CHART] 2. PARAMETER SPACE EXPLORATION ANALYSIS
----------------------------------------
   - Found 12 hyperparameters: ['params_batch

## Section 5: Final Model Comparison and Best-of-Best Selection

### 5.1 Re-run of models with best hyperparameters identified from Section 4

#### 5.1.1 Best CTGAN Model Evaluation

In [27]:
# Code Chunk ID: CHUNK_5_1_1_A
# Section 5.1: Best CTGAN Model Evaluation  
print("🏆 SECTION 5.1: BEST CTGAN MODEL EVALUATION")
print("=" * 60)

# ============================================================================
# LOAD BEST PARAMETERS FROM SECTION 4 (CSV + MEMORY FALLBACK)
# ============================================================================
print("📖 5.1.0 Loading best parameters from Section 4...")

try:
    # Load all best parameters using setup.py function
    param_data = load_best_parameters_from_csv(
        section_number=4,
        dataset_identifier=DATASET_IDENTIFIER,
        fallback_to_memory=True,
        scope=globals()
    )
    
    print(f"✅ Parameter loading completed from {param_data['source']}")
    print(f"   • Models available: {param_data['models_count']}")
    
    # Extract CTGAN parameters specifically
    loaded_ctgan_params = param_data['parameters'].get('ctgan', None)
    
except Exception as e:
    print(f"⚠️  Parameter loading failed: {str(e)}")
    print(f"   Falling back to direct memory access")
    loaded_ctgan_params = None

# 5.1.1 Retrieve Best Model Results from Section 4.1
print("\n📊 5.1.1 Retrieving best CTGAN results from Section 4.1...")

try:
    # Primary: Use loaded parameters if available
    if loaded_ctgan_params is not None:
        print(f"✅ Using loaded CTGAN parameters from {param_data['source']}")
        best_params = loaded_ctgan_params
        
        # Try to get additional metadata from memory if available
        if 'ctgan_study' in globals() and ctgan_study is not None and hasattr(ctgan_study, 'best_trial'):
            best_trial = ctgan_study.best_trial
            best_value = best_trial.value
            trial_number = best_trial.number
        else:
            # Use fallback values when memory unavailable  
            best_value = 0.0  # Will be recalculated during evaluation
            trial_number = "loaded_from_csv"
            print(f"   ⚠️  Memory study unavailable - using loaded parameters only")
        
    else:
        # Fallback: Direct memory access
        print(f"🔄 Falling back to direct memory access...")
        best_trial = ctgan_study.best_trial
        best_params = best_trial.params
        best_value = best_trial.value
        trial_number = best_trial.number
        print(f"✅ Using CTGAN parameters from memory")
    
    print(f"\n✅ Section 4.1 CTGAN optimization parameters retrieved!")
    print(f"   • Best Trial: #{trial_number}")
    print(f"   • Best Objective Score: {best_value:.4f}" if isinstance(best_value, (int, float)) else f"   • Best Objective Score: {best_value}")
    print(f"   • Parameter count: {len(best_params)}")
    
    # Display parameters
    print(f"\n📈 5.1.2 Best CTGAN configuration:")
    for param, value in best_params.items():
        if isinstance(value, float):
            print(f"   • {param}: {value:.4f}")
        else:
            print(f"   • {param}: {value}")
    
    print(f"🔍 Parameter source: {param_data.get('source', 'memory') if loaded_ctgan_params else 'memory'}")
    
    # ============================================================================
    # 5.1.3 TRAIN FINAL CTGAN MODEL WITH OPTIMIZED PARAMETERS
    # ============================================================================
    
    print(f"\n🔧 5.1.3 Training final CTGAN model with optimized parameters...")
    
    try:
        # Use ModelFactory pattern
        from src.models.model_factory import ModelFactory
        
        # Create CTGAN model
        final_ctgan_model = ModelFactory.create("ctgan", random_state=42)
        
        # Apply best parameters with defaults for missing values
        final_ctgan_params = {
            'epochs': best_params.get('epochs', 300),
            'batch_size': best_params.get('batch_size', 500),
            'generator_lr': best_params.get('generator_lr', 2e-4),
            'discriminator_lr': best_params.get('discriminator_lr', 2e-4),
            'generator_decay': best_params.get('generator_decay', 1e-6),
            'discriminator_decay': best_params.get('discriminator_decay', 1e-6),
            'pac': best_params.get('pac', 10),
            'verbose': best_params.get('verbose', True)
        }
        
        print("🔧 Training CTGAN with optimal hyperparameters...")
        for param, value in final_ctgan_params.items():
            print(f"   • Using {param}: {value}")
        
        # Train the model
        final_ctgan_model.train(data, **final_ctgan_params)
        print("✅ CTGAN training completed successfully!")
        
        # Generate synthetic data
        print("🎲 Generating synthetic data...")
        synthetic_ctgan_final = final_ctgan_model.generate(len(data))
        print(f"✅ Generated {len(synthetic_ctgan_final)} synthetic samples")
        
        # ============================================================================
        # 5.1.4 EVALUATE FINAL CTGAN MODEL PERFORMANCE
        # ============================================================================
        
        print("\n📊 5.1.4 Final CTGAN Model Evaluation...")
        
        # Use enhanced objective function for evaluation
        if 'enhanced_objective_function_v2' in globals():
            print("🎯 Enhanced objective function evaluation:")
            
            ctgan_final_score, ctgan_similarity, ctgan_accuracy = enhanced_objective_function_v2(
                real_data=data, 
                synthetic_data=synthetic_ctgan_final, 
                target_column=TARGET_COLUMN
            )
            
            print(f"\n✅ Final CTGAN Evaluation Results:")
            print(f"   • Overall Score: {ctgan_final_score:.4f}")
            print(f"   • Similarity Score: {ctgan_similarity:.4f} (60% weight)")  
            print(f"   • Accuracy Score: {ctgan_accuracy:.4f} (40% weight)")
            
            # Store results for Section 5.7 comparison
            ctgan_final_results = {
                'model_name': 'CTGAN',
                'objective_score': ctgan_final_score,
                'similarity_score': ctgan_similarity,
                'accuracy_score': ctgan_accuracy,
                'best_params': best_params,
                'parameter_source': param_data.get('source', 'memory') if loaded_ctgan_params else 'memory',
                'synthetic_data': synthetic_ctgan_final
            }
            
            print("🎯 CTGAN Final Assessment:")
            print(f"   • Production Ready: {'✅ Yes' if ctgan_final_score > 0.6 else '⚠️ Review Required'}")
            print(f"   • Recommended for: General-purpose tabular synthetic data generation")
            print(f"   • Final Score vs Optimization Score: {ctgan_final_score:.4f} vs {best_value:.4f}" if isinstance(best_value, (int, float)) else f"   • Final Score: {ctgan_final_score:.4f}")
            
        else:
            print("⚠️ Enhanced objective function not available - using basic evaluation")
            ctgan_final_results = {
                'model_name': 'CTGAN',
                'objective_score': best_value if isinstance(best_value, (int, float)) else 0.0,
                'best_params': best_params,
                'parameter_source': param_data.get('source', 'memory') if loaded_ctgan_params else 'memory',
                'synthetic_data': synthetic_ctgan_final
            }
                
    except Exception as train_error:
        print(f"❌ Failed to train final CTGAN model: {train_error}")
        import traceback
        traceback.print_exc()
        synthetic_ctgan_final = None
        ctgan_final_score = 0.0
        ctgan_final_results = {
            'model_name': 'CTGAN',
            'objective_score': 0.0,
            'error': str(train_error)
        }

except Exception as e:
    print(f"❌ Error accessing CTGAN parameters: {e}")
    print("   Please ensure Section 4.1 has been executed successfully or parameter CSV exists.")
    # Create empty results to prevent downstream errors
    synthetic_ctgan_final = None
    ctgan_final_results = {
        'model_name': 'CTGAN',
        'objective_score': 0.0,
        'error': str(e)
    }
    
print("\n" + "=" * 60)
print("✅ SECTION 5.1 COMPLETE: Best CTGAN model trained and evaluated")
print("🔄 Ready for Section 5.2: CTAB-GAN model training")

🏆 SECTION 5.1: BEST CTGAN MODEL EVALUATION
📖 5.1.0 Loading best parameters from Section 4...
[LOAD] LOADING BEST PARAMETERS FROM SECTION 4
[FOLDER] Looking for: results/pakistani-diabetes-dataset/2025-09-23/Section-4/best_parameters.csv
[OK] Found parameter CSV file
[OK] Loaded CTGAN: 12 parameters
[OK] Loaded CTAB-GAN: 3 parameters
[OK] Loaded CTAB-GAN+: 3 parameters
[OK] Loaded GANerAid: 11 parameters
[OK] Loaded CopulaGAN: 6 parameters
[OK] Loaded TVAE: 11 parameters

[LOAD] Parameter loading completed!
[SEARCH] Source: CSV file
[CHART] Models loaded: 6
   - ctgan: 12 parameters
   - ctabgan: 3 parameters
   - ctabganplus: 3 parameters
   - ganeraid: 11 parameters
   - copulagan: 6 parameters
   - tvae: 11 parameters
✅ Parameter loading completed from CSV file
   • Models available: 6

📊 5.1.1 Retrieving best CTGAN results from Section 4.1...
✅ Using loaded CTGAN parameters from CSV file

✅ Section 4.1 CTGAN optimization parameters retrieved!
   • Best Trial: #2
   • Best Objective 

Gen. (-1.07) | Discrim. (0.12): 100%|██████████| 450/450 [00:16<00:00, 27.27it/s] 


✅ CTGAN training completed successfully!
🎲 Generating synthetic data...
✅ Generated 500 synthetic samples

📊 5.1.4 Final CTGAN Model Evaluation...
🎯 Enhanced objective function evaluation:
[TARGET] Enhanced objective function using target column: 'Outcome'
[OK] Similarity Analysis: 19/19 valid metrics, Average: 0.5290
[OK] TRTS (Synthetic->Real): 0.5200
[OK] TRTS Evaluation: 2 scenarios, Average: 0.5240
[CHART] Combined Score: 0.5270 (Similarity: 0.5290, Accuracy: 0.5240)

✅ Final CTGAN Evaluation Results:
   • Overall Score: 0.5270
   • Similarity Score: 0.5290 (60% weight)
   • Accuracy Score: 0.5240 (40% weight)
🎯 CTGAN Final Assessment:
   • Production Ready: ⚠️ Review Required
   • Recommended for: General-purpose tabular synthetic data generation
   • Final Score vs Optimization Score: 0.5270 vs 0.7257

✅ SECTION 5.1 COMPLETE: Best CTGAN model trained and evaluated
🔄 Ready for Section 5.2: CTAB-GAN model training


#### 5.1.2 Best CTAB-GAN Model Evaluation

In [28]:
# Code Chunk ID: CHUNK_5_1_1_Aa

# Section 5.2: Best CTAB-GAN Model Evaluation
print("🏆 SECTION 5.2: BEST CTAB-GAN MODEL EVALUATION")
print("=" * 60)

# 5.2.1 Retrieve Best Model Results from Section 4.2
print("📊 5.2.1 Retrieving best CTAB-GAN results from Section 4.2...")

try:
    # Use unified parameter loading function
    ctabgan_params = get_model_parameters(
        model_name='ctab-gan',
        section_number=4,
        dataset_identifier=DATASET_IDENTIFIER,
        scope=globals()
    )
    
    if ctabgan_params is not None:
        best_params = ctabgan_params
        
        # Try to get additional metadata from memory if available
        if 'ctabgan_study' in globals() and ctabgan_study is not None:
            best_trial = ctabgan_study.best_trial
            best_objective_score = best_trial.value
            trial_number = best_trial.number
            print(f"✅ Section 4.2 CTAB-GAN optimization completed successfully!")
            print(f"   • Best Trial: #{trial_number}")
        else:
            # Use fallback values when memory unavailable
            best_objective_score = 0.0
            trial_number = "loaded_from_csv"
            print(f"✅ Section 4.2 CTAB-GAN parameters loaded from CSV!")
            print(f"   • Best Trial: #{trial_number}")
        
        print(f"   • Best Objective Score: {best_objective_score:.4f}" if isinstance(best_objective_score, (int, float)) else f"   • Best Objective Score: {best_objective_score}")
        print(f"   • Best Parameters:")
        for param, value in best_params.items():
            print(f"     - {param}: {value}")
        
        # 5.2.2 Train Final CTAB-GAN Model using Section 5.1 Pattern
        print("🔧 Training final CTAB-GAN model using Section 5.1 proven pattern with optimized parameters...")
        
        try:
            # Use the exact same ModelFactory pattern that works in Section 5.1
            from src.models.model_factory import ModelFactory
            
            # Create CTAB-GAN model using the working pattern
            final_ctabgan_model = ModelFactory.create("ctabgan", random_state=42)
            
            # Apply the best parameters found in Section 4.2 optimization
            final_ctabgan_params = {
                'epochs': best_params.get('epochs', 300),
                'batch_size': best_params.get('batch_size', 512),
                'lr': best_params.get('lr', 2e-4),
                'betas': best_params.get('betas', (0.5, 0.9)),
                'l2scale': best_params.get('l2scale', 1e-5),
                'mixed_precision': best_params.get('mixed_precision', False),
                'test_ratio': best_params.get('test_ratio', 0.20),
                'verbose': best_params.get('verbose', True)
            }
            
            print("🔧 Training CTAB-GAN with optimal hyperparameters...")
            for param, value in final_ctabgan_params.items():
                print(f"   • Using {param}: {value}")
            
            # Train the model with best parameters
            final_ctabgan_model.train(data, **final_ctabgan_params)
            print("✅ CTAB-GAN training completed successfully!")
            
            # Generate synthetic data
            print("📊 Generating synthetic data for evaluation...")
            synthetic_ctabgan_final = final_ctabgan_model.generate(len(data))
            print(f"✅ Generated {len(synthetic_ctabgan_final)} synthetic samples")
            
            # Evaluate using enhanced objective function
            if 'enhanced_objective_function_v2' in globals():
                print("🎯 CTAB-GAN Classification Performance Analysis:")
                
                ctabgan_final_score, ctabgan_similarity, ctabgan_accuracy = enhanced_objective_function_v2(
                    real_data=data, 
                    synthetic_data=synthetic_ctabgan_final, 
                    target_column=TARGET_COLUMN
                )
                
                print(f"✅ CTAB-GAN Final Results:")
                print(f"   • Overall Score: {ctabgan_final_score:.4f}")
                print(f"   • Similarity Score: {ctabgan_similarity:.4f}")  
                print(f"   • Accuracy Score: {ctabgan_accuracy:.4f}")
                
                # Store results for Section 5.7 comparison
                ctabgan_final_results = {
                    'model_name': 'CTAB-GAN',
                    'objective_score': ctabgan_final_score,
                    'similarity_score': ctabgan_similarity,
                    'accuracy_score': ctabgan_accuracy,
                    'best_params': best_params,
                    'synthetic_data': synthetic_ctabgan_final
                }
                
            else:
                print("⚠️ Enhanced objective function not available - using basic evaluation")
                ctabgan_final_results = {
                    'model_name': 'CTAB-GAN',
                    'objective_score': best_objective_score,
                    'best_params': best_params,
                    'synthetic_data': synthetic_ctabgan_final
                }
                
        except Exception as e:
            print(f"❌ CTAB-GAN training failed: {str(e)}")
            synthetic_ctabgan_final = None
            ctabgan_final_results = {
                'model_name': 'CTAB-GAN',
                'objective_score': 0.0,
                'error': str(e)
            }
        
    else:
        print("❌ CTAB-GAN study results not found - Section 4.2 may not have completed successfully")
        print("    Please ensure Section 4.2 has been executed before running Section 5.2")
        synthetic_ctabgan_final = None
        ctabgan_final_score = 0.0
        ctabgan_final_results = {
            'model_name': 'CTAB-GAN',
            'objective_score': 0.0,
            'error': 'Section 4.2 not completed'
        }
        
except Exception as e:
    print(f"❌ Error in Section 5.2 CTAB-GAN evaluation: {e}")
    import traceback
    traceback.print_exc()
    synthetic_ctabgan_final = None
    ctabgan_final_score = 0.0
    ctabgan_final_results = {
        'model_name': 'CTAB-GAN',
        'objective_score': 0.0,
        'error': str(e)
    }

print("✅ Section 5.2 CTAB-GAN evaluation completed!")
print("=" * 60)

🏆 SECTION 5.2: BEST CTAB-GAN MODEL EVALUATION
📊 5.2.1 Retrieving best CTAB-GAN results from Section 4.2...
[LOAD] LOADING BEST PARAMETERS FROM SECTION 4
[FOLDER] Looking for: results/pakistani-diabetes-dataset/2025-09-23/Section-4/best_parameters.csv
[OK] Found parameter CSV file
[OK] Loaded CTGAN: 12 parameters
[OK] Loaded CTAB-GAN: 3 parameters
[OK] Loaded CTAB-GAN+: 3 parameters
[OK] Loaded GANerAid: 11 parameters
[OK] Loaded CopulaGAN: 6 parameters
[OK] Loaded TVAE: 11 parameters

[LOAD] Parameter loading completed!
[SEARCH] Source: CSV file
[CHART] Models loaded: 6
   - ctgan: 12 parameters
   - ctabgan: 3 parameters
   - ctabganplus: 3 parameters
   - ganeraid: 11 parameters
   - copulagan: 6 parameters
   - tvae: 11 parameters
[OK] CTAB-GAN parameters loaded from CSV file
✅ Section 4.2 CTAB-GAN optimization completed successfully!
   • Best Trial: #2
   • Best Objective Score: 0.9570
   • Best Parameters:
     - epochs: 1000
     - batch_size: 64
     - test_ratio: 0.15
🔧 Traini

100%|██████████| 1000/1000 [06:12<00:00,  2.68it/s]


Finished training in 373.7130243778229  seconds.
✅ CTAB-GAN training completed successfully!
📊 Generating synthetic data for evaluation...
✅ Generated 500 synthetic samples
🎯 CTAB-GAN Classification Performance Analysis:
[TARGET] Enhanced objective function using target column: 'Outcome'
[OK] Similarity Analysis: 19/19 valid metrics, Average: 0.6647
[OK] TRTS (Synthetic->Real): 0.9900
[OK] TRTS Evaluation: 2 scenarios, Average: 0.9730
[CHART] Combined Score: 0.7880 (Similarity: 0.6647, Accuracy: 0.9730)
✅ CTAB-GAN Final Results:
   • Overall Score: 0.7880
   • Similarity Score: 0.6647
   • Accuracy Score: 0.9730
✅ Section 5.2 CTAB-GAN evaluation completed!


#### 5.1.3 Best CTAB-GAN+ Model Evaluation

In [29]:
# Code Chunk ID: CHUNK_5_1_3_A
# ============================================================================
# Section 5.3: Best CTAB-GAN+ Model Evaluation - FIXED IMPLEMENTATION
# ============================================================================
# Using Section 4.3 optimized hyperparameters with proven ModelFactory pattern

print("🏆 SECTION 5.3: BEST CTAB-GAN+ MODEL EVALUATION")
print("=" * 80)

try:
    # Step 1: Retrieve Section 4.3 CTAB-GAN+ optimization results
    if 'ctabganplus_study' in globals():
        best_trial = ctabganplus_study.best_trial
        best_params = best_trial.params
        best_objective_score = best_trial.value
        
        print(f"✅ Retrieved Section 4.3 CTAB-GAN+ optimization results")
        print(f"   • Best Trial: #{best_trial.number}")
        print(f"   • Best Objective Score: {best_objective_score:.4f}")
        print(f"   • Parameters: {len(best_params)} hyperparameters")
        
        # Display best parameters
        print(f"\n📊 Best CTAB-GAN+ Hyperparameters:")
        print("-" * 40)
        for param, value in best_params.items():
            if isinstance(value, float):
                print(f"   • {param}: {value:.4f}")
            else:
                print(f"   • {param}: {value}")
                
    else:
        print("⚠️ CTAB-GAN+ optimization results not found - using fallback parameters")
        # Fallback CTAB-GAN+ parameters (basic working configuration)
        best_params = {
            'epochs': 100,
            'batch_size': 128,
            'lr_generator': 1e-4,
            'lr_discriminator': 2e-4,
            'beta_1': 0.5,
            'beta_2': 0.9,
            'lambda_gp': 10,
            'pac': 1
        }
        best_objective_score = None
        print(f"   Using fallback parameters: {best_params}")

    # Step 2: Create CTAB-GAN+ model using proven ModelFactory pattern (SAME AS SECTION 5.2)
    print(f"\n🏗️ Creating CTAB-GAN+ model using ModelFactory...")
    from src.models.model_factory import ModelFactory
    
    # CRITICAL FIX: Use the exact same ModelFactory pattern that works in Section 5.1 & 5.2
    final_ctabganplus_model = ModelFactory.create("ctabganplus", random_state=42)
    print(f"✅ CTAB-GAN+ model created successfully")
    
    # Step 3: Train using the correct method name: .train() (NOT .fit())
    print(f"\n🚀 Training CTAB-GAN+ model with optimized hyperparameters...")
    print(f"   • Data shape: {data.shape}")
    print(f"   • Target column: '{TARGET_COLUMN}'")
    print(f"   • Training with Section 4.3 parameters")
    
    # Store final parameters for results tracking
    final_ctabganplus_params = best_params.copy()
    
    # CRITICAL FIX: Train using .train() method (proven pattern from Sections 5.1 & 5.2)
    final_ctabganplus_model.train(data, **final_ctabganplus_params)
    print(f"✅ CTAB-GAN+ model training completed successfully!")
    
    # Step 4: Generate synthetic data using the correct method: .generate()
    print(f"\n📊 Generating synthetic data for evaluation...")
    synthetic_ctabganplus_final = final_ctabganplus_model.generate(len(data))
    print(f"✅ Synthetic data generated successfully!")
    print(f"   • Synthetic data shape: {synthetic_ctabganplus_final.shape}")
    print(f"   • Columns match: {list(synthetic_ctabganplus_final.columns) == list(data.columns)}")
    
    # Step 5: Quick evaluation using enhanced objective function (NO IMPORT - function in globals)
    if 'enhanced_objective_function_v2' in globals():
        ctabganplus_final_score, ctabganplus_similarity, ctabganplus_accuracy = enhanced_objective_function_v2(
            real_data=data, 
            synthetic_data=synthetic_ctabganplus_final, 
            target_column=TARGET_COLUMN
        )
        
        print(f"\n📊 CTAB-GAN+ Enhanced Objective Function v2 Results:")
        print(f"   • Final Combined Score: {ctabganplus_final_score:.4f}")
        print(f"   • Statistical Similarity (60%): {ctabganplus_similarity:.4f}")
        print(f"   • Classification Accuracy (40%): {ctabganplus_accuracy:.4f}")
    else:
        print("⚠️ Enhanced objective function not available - using basic metrics")
        ctabganplus_final_score = 0.5  # Fallback score
        ctabganplus_similarity = 0.5
        ctabganplus_accuracy = 0.5
    
    # Store results for Section 5.7 comparative analysis
    ctabganplus_final_results = {
        'model_name': 'CTAB-GAN+',
        'objective_score': ctabganplus_final_score,
        'similarity_score': ctabganplus_similarity,
        'accuracy_score': ctabganplus_accuracy,
        'final_combined_score': ctabganplus_final_score,
        'sections_completed': ['5.3.1'],
        'evaluation_method': 'section_5_1_pattern',
        'section_4_optimization': best_objective_score is not None,
        'best_section_4_score': best_objective_score
    }
    
    print(f"\n✅ SECTION 5.3 COMPLETED SUCCESSFULLY!")
    print(f"🎯 CTAB-GAN+ evaluation completed using Section 4.3 optimized parameters")
    print(f"📊 Results ready for Section 5.7 comparative analysis")
    print("-" * 80)

except Exception as e:
    print(f"❌ CTAB-GAN+ evaluation failed: {str(e)}")
    import traceback
    traceback.print_exc()
    # Set fallback for subsequent sections
    synthetic_ctabganplus_final = None
    ctabganplus_final_results = {'error': str(e), 'evaluation_failed': True}

🏆 SECTION 5.3: BEST CTAB-GAN+ MODEL EVALUATION
✅ Retrieved Section 4.3 CTAB-GAN+ optimization results
   • Best Trial: #3
   • Best Objective Score: 0.9770
   • Parameters: 3 hyperparameters

📊 Best CTAB-GAN+ Hyperparameters:
----------------------------------------
   • epochs: 550
   • batch_size: 64
   • test_ratio: 0.2500

🏗️ Creating CTAB-GAN+ model using ModelFactory...
✅ CTAB-GAN+ model created successfully

🚀 Training CTAB-GAN+ model with optimized hyperparameters...
   • Data shape: (500, 19)
   • Target column: 'Outcome'
   • Training with Section 4.3 parameters


100%|██████████| 550/550 [04:47<00:00,  1.91it/s]


Finished training in 288.6458487510681  seconds.
✅ CTAB-GAN+ model training completed successfully!

📊 Generating synthetic data for evaluation...
✅ Synthetic data generated successfully!
   • Synthetic data shape: (500, 19)
   • Columns match: True
[TARGET] Enhanced objective function using target column: 'Outcome'
[OK] Similarity Analysis: 19/19 valid metrics, Average: 0.5407
[OK] TRTS (Synthetic->Real): 0.9820
[OK] TRTS Evaluation: 2 scenarios, Average: 0.9700
[CHART] Combined Score: 0.7124 (Similarity: 0.5407, Accuracy: 0.9700)

📊 CTAB-GAN+ Enhanced Objective Function v2 Results:
   • Final Combined Score: 0.7124
   • Statistical Similarity (60%): 0.5407
   • Classification Accuracy (40%): 0.9700

✅ SECTION 5.3 COMPLETED SUCCESSFULLY!
🎯 CTAB-GAN+ evaluation completed using Section 4.3 optimized parameters
📊 Results ready for Section 5.7 comparative analysis
--------------------------------------------------------------------------------


#### Section 5.1.4 BEST GANerAid MODEL

In [30]:
# Code Chunk ID: CHUNK_5_1_4_A
# ============================================================================
# Section 5.4.1: Best GANerAid Model Training
# ============================================================================
# Using Section 4.4 optimized hyperparameters with proven ModelFactory pattern

print("🏆 SECTION 5.4.1: BEST GANerAid MODEL TRAINING")
print("=" * 80)

try:
    # Step 1: Retrieve Section 4.4 GANerAid optimization results
    if 'ganeraid_study' in globals():
        best_trial = ganeraid_study.best_trial
        final_ganeraid_params = best_trial.params
        best_objective_score = best_trial.value
        
        print(f"✅ Retrieved Section 4.4 GANerAid optimization results")
        print(f"   • Best Trial: #{best_trial.number}")
        print(f"   • Best Objective Score: {best_objective_score:.4f}")
        print(f"   • Parameters: {len(final_ganeraid_params)} hyperparameters")
        
    else:
        print("⚠️ GANerAid optimization results not found - using fallback parameters")
        # Fallback GANerAid parameters
        final_ganeraid_params = {
            'epochs': 100,
            'batch_size': 128,
            'learning_rate': 1e-4
        }
        best_objective_score = None

    # Step 2: Create GANerAid model using proven ModelFactory pattern
    print(f"\n🏗️ Creating GANerAid model using ModelFactory...")
    from src.models.model_factory import ModelFactory
    
    final_ganeraid_model = ModelFactory.create("ganeraid", random_state=42)
    print(f"✅ GANerAid model created successfully")
    
    # Step 3: Train using .train() method (NOT .fit())
    print(f"\n🚀 Training GANerAid model with optimized hyperparameters...")
    final_ganeraid_model.train(data, **final_ganeraid_params)
    print(f"✅ GANerAid model training completed successfully!")
    
    # Step 4: Generate synthetic data
    synthetic_ganeraid_final = final_ganeraid_model.generate(len(data))
    print(f"✅ GANerAid synthetic data generated: {synthetic_ganeraid_final.shape}")
    
    # Step 5: Quick evaluation using enhanced objective function (NO IMPORT - function in globals)
    if 'enhanced_objective_function_v2' in globals():
        ganeraid_final_score, ganeraid_similarity, ganeraid_accuracy = enhanced_objective_function_v2(
            real_data=data, synthetic_data=synthetic_ganeraid_final, target_column=TARGET_COLUMN
        )
        
        print(f"\n📊 GANerAid Enhanced Objective Function v2 Results:")
        print(f"   • Final Combined Score: {ganeraid_final_score:.4f}")
        print(f"   • Statistical Similarity (60%): {ganeraid_similarity:.4f}")
        print(f"   • Classification Accuracy (40%): {ganeraid_accuracy:.4f}")
    else:
        print("⚠️ Enhanced objective function not available - using basic metrics")
        ganeraid_final_score = 0.5  # Fallback score
        ganeraid_similarity = 0.5
        ganeraid_accuracy = 0.5
    
    # Store results
    ganeraid_final_results = {
        'model_name': 'GANerAid',
        'objective_score': ganeraid_final_score,
        'similarity_score': ganeraid_similarity,
        'accuracy_score': ganeraid_accuracy,
        'final_combined_score': ganeraid_final_score,
        'sections_completed': ['5.4.1'],
        'evaluation_method': 'section_5_1_pattern',
        'section_4_optimization': best_objective_score is not None,
        'best_section_4_score': best_objective_score,
        'optimized_params': final_ganeraid_params
    }
    
    print(f"\n✅ SECTION 5.4.1 - GANerAid MODEL TRAINING COMPLETED!")
    print("-" * 80)

except Exception as e:
    print(f"❌ GANerAid training failed: {str(e)}")
    import traceback
    traceback.print_exc()
    synthetic_ganeraid_final = None
    ganeraid_final_results = {'error': str(e), 'training_failed': True}

WARNING	src.models.implementations.ganeraid_model:ganeraid_model.py:train()- nr_of_rows adjusted for dimension compatibility: 18 -> 4


🏆 SECTION 5.4.1: BEST GANerAid MODEL TRAINING
✅ Retrieved Section 4.4 GANerAid optimization results
   • Best Trial: #4
   • Best Objective Score: 0.5576
   • Parameters: 11 hyperparameters

🏗️ Creating GANerAid model using ModelFactory...
✅ GANerAid model created successfully

🚀 Training GANerAid model with optimized hyperparameters...
Initialized gan with the following parameters: 
lr_d = 0.001857992893806313
lr_g = 0.00013956404896338346
hidden_feature_space = 500
batch_size = 256
nr_of_rows = 4
binary_noise = 0.05206226298620407
Start training of gan for 250 epochs


100%|██████████| 250/250 [01:45<00:00,  2.36it/s, loss=d error: 0.7026879489421844 --- g error 2.0046963691711426]  


✅ GANerAid model training completed successfully!
Generating 500 samples
✅ GANerAid synthetic data generated: (500, 19)
[TARGET] Enhanced objective function using target column: 'Outcome'
[OK] Similarity Analysis: 19/19 valid metrics, Average: 0.5262
[OK] TRTS (Synthetic->Real): 0.8740
[OK] TRTS Evaluation: 2 scenarios, Average: 0.7530
[CHART] Combined Score: 0.6169 (Similarity: 0.5262, Accuracy: 0.7530)

📊 GANerAid Enhanced Objective Function v2 Results:
   • Final Combined Score: 0.6169
   • Statistical Similarity (60%): 0.5262
   • Classification Accuracy (40%): 0.7530

✅ SECTION 5.4.1 - GANerAid MODEL TRAINING COMPLETED!
--------------------------------------------------------------------------------


#### 5.1.5: Best CopulaGAN Model

In [31]:
# Code Chunk ID: CHUNK_5_1_5_A
# ============================================================================
# Section 5.5: Best CopulaGAN Model Evaluation
# ============================================================================
# Using Section 4.5 optimized hyperparameters with proven ModelFactory pattern

# Import ModelFactory for CopulaGAN creation
from src.models.model_factory import ModelFactory

print("🎯 ============================================================================")
print("🎯 Section 5.5: CopulaGAN Final Model Training & Evaluation")
print("🎯 ============================================================================")

try:
    # Load all best parameters using setup.py function
    param_data = load_best_parameters_from_csv(
        section_number=4,
        dataset_identifier=DATASET_IDENTIFIER,
        fallback_to_memory=True,
        scope=globals()
    )
    
    # Extract CopulaGAN parameters specifically
    loaded_copulagan_params = param_data['parameters'].get('copulagan', None)
    
    if loaded_copulagan_params:
        print(f"📋 Loaded CopulaGAN parameters from {param_data.get('source', 'unknown')}:")
        for param, value in loaded_copulagan_params.items():
            print(f"   • {param}: {value}")
        # Use all optimized parameters from Section 4.5
        best_params = loaded_copulagan_params.copy()
        param_source = param_data.get('source', 'CSV')
    else:
        print("⚠️ CopulaGAN optimization results not found - using fallback parameters")
        best_params = {'epochs': 500, 'batch_size': 100}  # Use Section 3 proven values
        param_source = 'fallback'
    
    print(f"\n🔧 Preprocessing data for CopulaGAN...")
    data_preprocessed = data.copy()
    print(f"   ✅ Data preprocessing completed: {data_preprocessed.shape}")
    print(f"   • Missing values: {data_preprocessed.isnull().sum().sum()}")
    
    # Show data types
    dtype_counts = data_preprocessed.dtypes.value_counts()
    print(f"   • Data types: {dict(dtype_counts)}")
    
    print(f"\n🏗️ Creating CopulaGAN model using ModelFactory...")
    final_copulagan_model = ModelFactory.create("copulagan", random_state=42)
    print("✅ CopulaGAN model created successfully")
    
    print(f"\n🚀 Training CopulaGAN model with optimized hyperparameters...")
    print(f"   • Using parameters: {best_params}")
    
    # Train the model with all optimized parameters
    final_copulagan_model.train(data_preprocessed, **best_params)
    
    print("✅ CopulaGAN model training completed successfully")
    
    # Generate synthetic data
    print(f"\n🎲 Generating {len(data_preprocessed)} synthetic samples...")
    synthetic_copulagan_final = final_copulagan_model.generate(len(data_preprocessed))
    print(f"   ✅ Generated synthetic data: {synthetic_copulagan_final.shape}")
    
    # Comprehensive evaluation with enhanced objective function - using correct parameters
    print(f"\n📊 Comprehensive model evaluation...")
    copulagan_final_score, copulagan_similarity, copulagan_accuracy = enhanced_objective_function_v2(
        real_data=data_preprocessed,
        synthetic_data=synthetic_copulagan_final,
        target_column=TARGET_COLUMN
    )
    
    print(f"\n🎯 === CopulaGAN Final Results (Section 5.5) ===")
    print(f"📈 Combined Score: {copulagan_final_score:.4f}")
    print(f"📊 Statistical Similarity: {copulagan_similarity:.4f}")
    print(f"🎯 Predictive Accuracy: {copulagan_accuracy:.4f}")
    print(f"🔧 Parameter Source: {param_source}")
    
    # Store results for comparison
    copulagan_final_result = {
        'model_name': 'CopulaGAN_Final',
        'combined_score': copulagan_final_score,
        'similarity_score': copulagan_similarity,
        'accuracy_score': copulagan_accuracy,
        'hyperparameters': best_params,
        'parameter_source': param_source
    }
    
    print("✅ CopulaGAN final model evaluation completed successfully")
    
except Exception as e:
    print(f"❌ Error in CopulaGAN final model evaluation: {str(e)}")
    import traceback
    traceback.print_exc()
    
    # Fallback result to prevent pipeline failure
    copulagan_final_result = {
        'model_name': 'CopulaGAN_Final',
        'combined_score': 0.0,
        'similarity_score': 0.0,
        'accuracy_score': 0.0,
        'hyperparameters': {},
        'parameter_source': 'error_fallback'
    }

🎯 ============================================================================
🎯 Section 5.5: CopulaGAN Final Model Training & Evaluation
🎯 ============================================================================
[LOAD] LOADING BEST PARAMETERS FROM SECTION 4
[FOLDER] Looking for: results/pakistani-diabetes-dataset/2025-09-23/Section-4/best_parameters.csv
[OK] Found parameter CSV file
[OK] Loaded CTGAN: 12 parameters
[OK] Loaded CTAB-GAN: 3 parameters
[OK] Loaded CTAB-GAN+: 3 parameters
[OK] Loaded GANerAid: 11 parameters
[OK] Loaded CopulaGAN: 6 parameters
[OK] Loaded TVAE: 11 parameters

[LOAD] Parameter loading completed!
[SEARCH] Source: CSV file
[CHART] Models loaded: 6
   - ctgan: 12 parameters
   - ctabgan: 3 parameters
   - ctabganplus: 3 parameters
   - ganeraid: 11 parameters
   - copulagan: 6 parameters
   - tvae: 11 parameters
📋 Loaded CopulaGAN parameters from CSV file:
   • epochs: 500
   • batch_size: 200
   • generator_lr: 0.003930639677378101
   • discriminator_lr: 

#### 5.1.6: Best TVAE Model Evaluation 

In [32]:
# Code Chunk ID: CHUNK_5_1_6_A
# ============================================================================
# Section 5.6.1: Best TVAE Model Training
# ============================================================================
# Using Section 4.6 optimized hyperparameters with proven ModelFactory pattern

print("🏆 SECTION 5.6.1: BEST TVAE MODEL TRAINING")
print("=" * 80)

try:
    # Step 1: Retrieve Section 4.6 TVAE optimization results
    if 'tvae_study' in globals():
        best_trial = tvae_study.best_trial
        final_tvae_params = best_trial.params
        best_objective_score = best_trial.value
        
        print(f"✅ Retrieved Section 4.6 TVAE optimization results")
        print(f"   • Best Trial: #{best_trial.number}")
        print(f"   • Best Objective Score: {best_objective_score:.4f}")
        print(f"   • Parameters: {len(final_tvae_params)} hyperparameters")
        
    else:
        print("⚠️ TVAE optimization results not found - using fallback parameters")
        # Fallback TVAE parameters
        final_tvae_params = {
            'epochs': 100,
            'batch_size': 128,
            'lr': 1e-4,
            'compress_dims': [128, 64],
            'decompress_dims': [64, 128]
        }
        best_objective_score = None

    # Step 2: Create TVAE model using proven ModelFactory pattern
    print(f"\n🏗️ Creating TVAE model using ModelFactory...")
    from src.models.model_factory import ModelFactory
    
    final_tvae_model = ModelFactory.create("tvae", random_state=42)
    print(f"✅ TVAE model created successfully")
    
    # Step 3: Train using .train() method (NOT .fit())
    print(f"\n🚀 Training TVAE model with optimized hyperparameters...")
    final_tvae_model.train(data, **final_tvae_params)
    print(f"✅ TVAE model training completed successfully!")
    
    # Step 4: Generate synthetic data
    synthetic_tvae_final = final_tvae_model.generate(len(data))
    print(f"✅ TVAE synthetic data generated: {synthetic_tvae_final.shape}")
    
    # Step 5: Quick evaluation using enhanced objective function (NO IMPORT - function in globals)
    if 'enhanced_objective_function_v2' in globals():
        tvae_final_score, tvae_similarity, tvae_accuracy = enhanced_objective_function_v2(
            real_data=data, synthetic_data=synthetic_tvae_final, target_column=TARGET_COLUMN
        )
        
        print(f"\n📊 TVAE Enhanced Objective Function v2 Results:")
        print(f"   • Final Combined Score: {tvae_final_score:.4f}")
        print(f"   • Statistical Similarity (60%): {tvae_similarity:.4f}")
        print(f"   • Classification Accuracy (40%): {tvae_accuracy:.4f}")
    else:
        print("⚠️ Enhanced objective function not available - using basic metrics")
        tvae_final_score = 0.5  # Fallback score
        tvae_similarity = 0.5
        tvae_accuracy = 0.5
    
    # Store results
    tvae_final_results = {
        'model_name': 'TVAE',
        'objective_score': tvae_final_score,
        'similarity_score': tvae_similarity,
        'accuracy_score': tvae_accuracy,
        'final_combined_score': tvae_final_score,
        'sections_completed': ['5.6.1'],
        'evaluation_method': 'section_5_1_pattern',
        'section_4_optimization': best_objective_score is not None,
        'best_section_4_score': best_objective_score,
        'optimized_params': final_tvae_params
    }
    
    print(f"\n✅ SECTION 5.6.1 - TVAE MODEL TRAINING COMPLETED!")
    print("-" * 80)

except Exception as e:
    print(f"❌ TVAE training failed: {str(e)}")
    import traceback
    traceback.print_exc()
    synthetic_tvae_final = None
    tvae_final_results = {'error': str(e), 'training_failed': True}

🏆 SECTION 5.6.1: BEST TVAE MODEL TRAINING
✅ Retrieved Section 4.6 TVAE optimization results
   • Best Trial: #2
   • Best Objective Score: 0.7587
   • Parameters: 11 hyperparameters

🏗️ Creating TVAE model using ModelFactory...
✅ TVAE model created successfully

🚀 Training TVAE model with optimized hyperparameters...
✅ TVAE model training completed successfully!
✅ TVAE synthetic data generated: (500, 19)
[TARGET] Enhanced objective function using target column: 'Outcome'
[OK] Similarity Analysis: 19/19 valid metrics, Average: 0.6002
[OK] TRTS (Synthetic->Real): 0.9620
[OK] TRTS Evaluation: 2 scenarios, Average: 0.9440
[CHART] Combined Score: 0.7377 (Similarity: 0.6002, Accuracy: 0.9440)

📊 TVAE Enhanced Objective Function v2 Results:
   • Final Combined Score: 0.7377
   • Statistical Similarity (60%): 0.6002
   • Classification Accuracy (40%): 0.9440

✅ SECTION 5.6.1 - TVAE MODEL TRAINING COMPLETED!
--------------------------------------------------------------------------------


### 5.2 Batch Process

This section outputs figures and graphics from models run in 54.1. The next chunk will output results for whatever subsections of 5.1 were run. I.e., if there's need to debug one method, there's no need to run all subsections of 5.1.

In [33]:
# Code Chunk ID: CHUNK_5_2_0_A
# ============================================================================
# SECTION 5.2 - OPTIMIZED MODELS BATCH EVALUATION
# Following CHUNK_018 pattern with comprehensive file export to Section-5 directory
# ============================================================================

print("🔍 SECTION 5.2 - OPTIMIZED MODELS BATCH EVALUATION")
print("=" * 80)
print("📋 Evaluating all available optimized models from Section 5.1.x")
print("📁 Exporting all tables and analysis to Section-5 directory")
print("🔄 Following Section 3 comprehensive evaluation pattern")
print()

# Ensure setup module function is available
from setup import evaluate_section5_optimized_models

# Use Section 5 batch evaluation function from setup.py
# Following exact same pattern as CHUNK_018 (Section 3) - comprehensive file export!
try:
    # Run batch evaluation with file export for all optimized models
    section5_batch_results = evaluate_section5_optimized_models(
        section_number=5,
        scope=globals(),  # Pass notebook scope to access synthetic data variables
        target_column=TARGET_COLUMN
    )
    
    print("\n" + "="*80)
    print("✅ SECTION 5.2 OPTIMIZED MODELS BATCH EVALUATION COMPLETED!")
    print("="*80)
    print(f"📊 Models processed: {section5_batch_results['models_processed']}")
    print(f"📁 Results exported to: {section5_batch_results['results_dir']}")
    
    # Show summary of all evaluations
    if 'evaluation_summaries' in section5_batch_results:
        print("\n📋 EVALUATION SUMMARIES:")
        print("-" * 40)
        for model_name, summary in section5_batch_results['evaluation_summaries'].items():
            print(f"🤖 {model_name}:")
            print(f"   📊 Synthetic samples: {summary.get('synthetic_samples', 'N/A')}")
            print(f"   📈 Overall score: {summary.get('overall_score', 'N/A')}")
            
    print("\n" + "="*80)
            
except Exception as e:
    print(f"❌ Section 5.2 batch evaluation failed: {e}")
    print(f"🔍 Error details: {type(e).__name__}")
    print()
    print("⚠️  Check that Section 5.1.x models completed successfully")

print("\n📈 Section 5.2 optimized model batch evaluation complete!")
print("🏁 Ready for final model comparison and production deployment!")

🔍 SECTION 5.2 - OPTIMIZED MODELS BATCH EVALUATION
📋 Evaluating all available optimized models from Section 5.1.x
📁 Exporting all tables and analysis to Section-5 directory
🔄 Following Section 3 comprehensive evaluation pattern

[SEARCH] UNIFIED BATCH EVALUATION - SECTION 5
[INFO] Dataset: pakistani-diabetes-dataset
[INFO] Target column: Outcome
[INFO] Variable pattern: final
[INFO] Found 6 trained models:
   [OK] CTGAN
   [OK] CTABGAN
   [OK] CTABGANPLUS
   [OK] GANerAid
   [OK] CopulaGAN
   [OK] TVAE

==================== EVALUATING CTGAN ====================
[SEARCH] CTGAN - COMPREHENSIVE DATA QUALITY EVALUATION
[FOLDER] Output directory: results\pakistani-diabetes-dataset\2025-09-23\Section-5\CTGAN

[1] STATISTICAL SIMILARITY
------------------------------
   [CHART] Average Statistical Similarity: 0.782

[2] PCA COMPARISON ANALYSIS WITH OUTCOME COLOR-CODING
--------------------------------------------------
   [CHART] PCA comparison plot saved: pca_comparison_with_outcome.png
   [C